# Imports

In [5]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
import sys
import dask as dd
from dask import dataframe
sys.path.insert(0, "C:/workspace/stress_test_sim//libs")

from monte_carlo import CauchyGenerator, PoissonGenerator, NormalGenerator, ParetoGenerator, LognormalGenerator, Transaction, MonteCarloTransactionSimulator, CauchyParameterSearcher, LognormalParameterSearcher
from uniswap_v2_extractor import get_pool_v2_reserves_history, get_pool_v2_history, list_to_transaction_dictionary, list_to_reserves_dictionary, pool_reserves_to_df, pool_history_to_df, get_pool_v2_mints, get_pool_v2_burns, list_to_mints_dictionary, pool_mints_to_df, pool_burns_to_df, filter_swaps
from dima_trubca_v2_plots import show_swaps_count_moving_averages, show_swaps_reserves_evolution_through_time, show_pool_price_evolution_from_reserves, show_swaps_amount_in_moving_averages

from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# WETH/USDC pool data

## Collecting WETH/USDC mints

In [2]:
eth_usdc_mints = get_pool_v2_mints(contract_id='0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc', range_limit=33)
eth_usdc_mints_df = pool_mints_to_df(eth_usdc_mints)

  6%|▌         | 2/33 [00:03<01:00,  1.94s/it]


KeyboardInterrupt: 

### writing received data to the pickle

In [9]:
eth_usdc_mints_df.to_pickle(os.getcwd() + "\\eth_usdc_mints.pkl")

### reading received data from pickle

In [ ]:
eth_usdc_mints_df = pd.read_pickle(os.getcwd() + "\\eth_usdc_mints.pkl")

## Collecting WETH/USDC burns

In [17]:
eth_usdc_burns = get_pool_v2_burns(contract_id='0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc', range_limit=27)
eth_usdc_burns_df = pool_burns_to_df(eth_usdc_burns)

 93%|█████████▎| 25/27 [00:30<00:02,  1.06s/it]

list index out of range


 96%|█████████▋| 26/27 [00:30<00:00,  1.21it/s]

list index out of range


100%|██████████| 27/27 [00:30<00:00,  1.15s/it]

list index out of range


In [18]:
eth_usdc_burns_df

,amount0,amount1,amountUSD,liquidity,timestamp
0,3.262000e-03,0.000016,3.263003e-03,2.299449e-10,2020-05-07 15:47:25
1,1.418900e-02,0.000070,1.419001e-02,9.999739e-10,2020-05-08 02:43:59
2,7.812840e-01,0.006539,7.812847e-01,6.960485e-08,2020-05-12 20:18:50
3,9.953090e-01,0.004114,8.361725e-01,6.217433e-08,2020-05-13 23:48:21
4,1.097105e+00,0.005132,1.047697e+00,7.289922e-08,2020-05-14 14:24:41
...,...,...,...,...,...
23307,2.613006e+03,0.543883,5.227160e+03,2.112073e-05,2021-11-09 10:28:54
23308,3.257345e+05,69.085029,6.515458e+05,2.656415e-03,2021-11-10 01:06:40
23309,1.090435e+06,229.948711,2.180152e+06,8.866823e-03,2021-11-10 02:42:27
23310,1.573420e+02,0.033234,3.146902e+02,1.280273e-06,2021-11-10 10:59:39


### Writing received data to pickle

In [19]:
eth_usdc_burns_df.to_pickle(os.getcwd() + "\\eth_usdc_burns.pkl")

### Reading received data from pickle

In [21]:
eth_usdc_burns_df = pd.read_pickle(os.getcwd() + "\\eth_usdc_burns.pkl")

## Plotting burns data

# WBTC/USDC pool data

## Collecting WBTC/USDC swaps, mints, burns, reserves

In [ ]:
wbtc_usdc_history = get_pool_v2_history(contract_id='0x004375dff511095cc5a197a54140a24efef3a416', range_limit=100)
all_wbtc_usdc_history, other_wbtc_usdc_history = filter_swaps(wbtc_usdc_history)
wbtc_usdc_df = pool_history_to_df(all_wbtc_usdc_history)
wbtc_usdc_df

In [15]:
wbtc_usdc_mints = get_pool_v2_mints(contract_id='0x004375dff511095cc5a197a54140a24efef3a416', range_limit=2)
wbtc_usdc_mints_df = pool_mints_to_df(wbtc_usdc_mints)
wbtc_usdc_mints_df

100%|██████████| 2/2 [00:01<00:00,  1.91it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,0.000058,0.565606,1.127788,5.641800e-14,2020-05-18 21:17:30
1,0.031843,308.949238,619.783548,3.136322e-11,2020-05-19 12:27:11
2,0.010943,106.173412,211.873034,1.077827e-11,2020-05-19 23:44:23
3,0.000016,0.152924,0.305054,1.551800e-14,2020-05-19 23:44:23
4,0.031445,305.317587,609.861677,3.098311e-11,2020-05-20 05:27:48
...,...,...,...,...,...
627,0.324345,15492.806311,30977.081673,5.993353e-10,2021-10-04 13:09:34
628,0.134754,6967.043706,13891.061970,2.587420e-10,2021-10-06 01:00:13
629,0.083782,4720.518454,9444.627005,1.676806e-10,2021-10-11 12:02:45
630,0.245886,15222.753672,30406.420741,5.147497e-10,2021-10-18 23:26:08


In [16]:
wbtc_usdc_burns = get_pool_v2_burns(contract_id='0x004375dff511095cc5a197a54140a24efef3a416', range_limit=2)
wbtc_usdc_burns_df = pool_burns_to_df(wbtc_usdc_burns)
wbtc_usdc_burns_df

100%|██████████| 2/2 [00:01<00:00,  1.96it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,0.000020,0.197013,0.392774,2.000000e-14,2020-05-18 21:20:11
1,0.063276,614.380910,1229.173591,6.234632e-11,2020-05-20 06:27:58
2,0.100240,926.246269,1878.633799,9.633865e-11,2020-05-28 22:37:40
3,0.034953,347.926154,685.842394,3.486130e-11,2020-06-05 16:08:49
4,0.010843,102.534555,203.973412,1.053852e-11,2020-06-21 12:23:07
...,...,...,...,...,...
474,0.082274,4841.457618,9693.518125,1.676806e-10,2021-10-27 11:34:40
475,0.142234,8724.538071,17461.585339,2.959248e-10,2021-10-28 14:59:15
476,0.422065,26708.491218,53304.471895,8.909240e-10,2021-11-03 11:18:10
477,0.007976,465.765580,934.462955,1.610082e-11,2021-11-20 12:31:42


In [17]:
wbtc_usdc_reserves = get_pool_v2_reserves_history(contract_id='0x004375dff511095cc5a197a54140a24efef3a416')
wbtc_usdc_reserves_df = pool_reserves_to_df(wbtc_usdc_reserves)
wbtc_usdc_reserves_df

100%|██████████| 2/2 [00:00<00:00,  2.01it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,0.000038,3.685930e-01,7.349464e-01,0.000000,0.000000,2020-05-18
1,0.042824,4.157976e+02,8.294344e+02,0.000016,0.153417,2020-05-19
2,0.010993,1.067343e+02,2.135402e+02,0.000000,0.000000,2020-05-20
3,0.011243,1.043669e+02,2.075372e+02,0.000250,2.367365,2020-05-21
4,0.112290,1.023060e+03,2.051484e+03,0.001047,9.603013,2020-05-22
...,...,...,...,...,...,...
466,18.025810,1.073007e+06,2.150214e+06,0.881248,51728.090515,2021-11-20
467,18.004379,1.072128e+06,2.138679e+06,5.747094,306366.572038,2021-11-21
468,18.540073,1.041546e+06,2.087521e+06,2.332608,133783.104812,2021-11-22
469,18.294648,1.055896e+06,2.106239e+06,2.190978,125684.620967,2021-11-23


### Writing swaps, mints, burns, reserves into pickle files

In [22]:
wbtc_usdc_df.to_pickle(os.getcwd() + "\\pkl_stories\\wbtc_usdc_swaps.pkl")
wbtc_usdc_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\wbtc_usdc_mints.pkl")
wbtc_usdc_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\wbtc_usdc_burns.pkl")
wbtc_usdc_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\wbtc_usdc_reserves.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [23]:
wbtc_usdc_df = pd.read_pickle(os.getcwd() + '\\pkl_stories\\wbtc_usdc_swaps.pkl')
wbtc_usdc_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\wbtc_usdc_mints.pkl")
wbtc_usdc_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\wbtc_usdc_burns.pkl")
wbtc_usdc_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\wbtc_usdc_reserves.pkl")

# WBTC/DAI pool data

## Collecting swaps, mints, burns and reserves

In [24]:
wbtc_dai_history = get_pool_v2_history(contract_id='0x231b7589426ffe1b75405526fc32ac09d44364c4', range_limit=20)
all_wbtc_dai_history, other_wbtc_dai_history = filter_swaps(wbtc_dai_history)
wbtc_dai_df = pool_history_to_df(all_wbtc_dai_history)
wbtc_dai_df

 75%|███████▌  | 15/20 [00:22<00:05,  1.09s/it]

list index out of range


 85%|████████▌ | 17/20 [00:22<00:01,  1.57it/s]

list index out of range
list index out of range


 90%|█████████ | 18/20 [00:23<00:01,  1.93it/s]

list index out of range


100%|██████████| 20/20 [00:23<00:00,  2.77it/s]

list index out of range
list index out of range


100%|██████████| 20/20 [00:23<00:00,  1.17s/it]


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,WBTC,DAI,0.000039,0.378642,0.378116,2020-05-20 11:55:36,0xfade503916c1d1253646c36c9961aa47bf14bd2d,0xfade503916c1d1253646c36c9961aa47bf14bd2d,0x337e7bf5b2b4c385597b11dffcf70d0cddac162d4772...
1,WBTC,DAI,0.000017,0.162868,0.161100,2020-05-20 18:34:15,0xfade503916c1d1253646c36c9961aa47bf14bd2d,0xfade503916c1d1253646c36c9961aa47bf14bd2d,0xd36f3e8bc306eb4c2ff894d1a8932c4647913af9d322...
2,WBTC,DAI,0.000256,2.450812,2.382097,2020-05-22 00:32:08,0x96569f124f6a3b62093c7115f14f8005705ba48a,0x96569f124f6a3b62093c7115f14f8005705ba48a,0xf3eaa95adf2319acf5b57db3f345b4a3f607bba4a833...
3,WBTC,DAI,0.000022,0.208767,0.206079,2020-05-22 10:27:44,0x96569f124f6a3b62093c7115f14f8005705ba48a,0x96569f124f6a3b62093c7115f14f8005705ba48a,0x5c08ee19949610beb1c54ae00eb576564a26a26cb029...
4,WBTC,DAI,0.000022,0.208199,0.205794,2020-05-22 10:30:29,0x96569f124f6a3b62093c7115f14f8005705ba48a,0x96569f124f6a3b62093c7115f14f8005705ba48a,0xb27b3c99425963ec06978be76fb6c47ce34eaf3fbb16...
...,...,...,...,...,...,...,...,...,...
13868,WBTC,DAI,0.068615,3894.961827,3899.793061,2021-11-23 11:25:14,0x51399b32cd0186bb32230e24167489f3b2f47870,0x51399b32cd0186bb32230e24167489f3b2f47870,0x83db84a6f7f8e41984d0683b247ca87f1580282eda75...
13869,DAI,WBTC,14.045500,0.000247,13.985478,2021-11-23 14:23:54,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xbb2b8038a1640196fbe3e38816f3e67cba72d940,0x6919e2bfe00c96d0951edb03ab9abbafbe48b493319b...
13870,DAI,WBTC,4999.996244,0.087407,4980.563327,2021-11-23 14:57:44,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xbb2b8038a1640196fbe3e38816f3e67cba72d940,0xc992ffe9f93e34b458f938c6303016dbbf7546af9219...
13871,WBTC,DAI,0.072017,4100.588141,4114.912375,2021-11-24 02:57:11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x3bd3ce01c82a12d7cff7c85a9e8bb27ae42fb548,0xcc334d82c5f46c8243584cf3014422235920e0f2a2bc...


In [25]:
wbtc_dai_reserves = get_pool_v2_reserves_history(contract_id='0x231b7589426ffe1b75405526fc32ac09d44364c4')
wbtc_dai_reserves_df = pool_reserves_to_df(wbtc_dai_reserves)
wbtc_dai_reserves_df

100%|██████████| 2/2 [00:00<00:00,  2.35it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,0.015897,156.421121,3.110206e+02,0.000000,0.000000,2020-05-18
1,0.015952,155.879611,3.080758e+02,0.000055,0.541510,2020-05-20
2,0.116257,1093.860346,2.163052e+03,0.000305,2.916207,2020-05-22
3,0.117786,1079.701885,2.147687e+03,0.001529,14.158461,2020-05-23
4,0.120951,1051.538677,2.108504e+03,0.003164,28.163208,2020-05-24
...,...,...,...,...,...,...
539,11.296106,671459.732196,1.343567e+06,0.297013,17373.290939,2021-11-20
540,11.311173,670600.826398,1.345993e+06,0.200406,11790.384560,2021-11-21
541,11.618130,652953.162706,1.307002e+06,0.410049,23604.694499,2021-11-22
542,11.490111,660302.414126,1.319415e+06,0.434642,24677.432070,2021-11-23


In [26]:
wbtc_dai_mints = get_pool_v2_mints(contract_id='0x231b7589426ffe1b75405526fc32ac09d44364c4', range_limit=1)
wbtc_dai_mints_df = pool_mints_to_df(wbtc_dai_mints)
wbtc_dai_mints_df

100%|██████████| 1/1 [00:00<00:00,  1.92it/s]


,amount0,amount1,amountUSD,liquidity,timestamp
0,0.000058,0.573588,1.138998,5.782254e-08,2020-05-18 21:18:56
1,0.015859,156.045929,310.274573,1.573075e-05,2020-05-18 22:22:40
2,0.100000,940.896942,1860.574893,9.699443e-05,2020-05-22 21:37:08
3,0.008735,77.283604,154.781806,8.215019e-06,2020-05-26 12:22:59
4,0.056679,501.500211,1000.979806,5.330799e-05,2020-05-26 22:37:46
...,...,...,...,...,...
182,1.225796,75652.079257,151603.780405,2.802349e-03,2021-10-16 05:47:00
183,1.434042,88504.336796,177438.237016,3.278430e-03,2021-10-16 06:03:22
184,0.648123,40000.000070,80079.267857,1.481704e-03,2021-10-16 06:25:30
185,0.394329,24999.999867,50076.234030,9.118558e-04,2021-11-13 12:42:30


In [27]:
wbtc_dai_burns = get_pool_v2_burns(contract_id='0x231b7589426ffe1b75405526fc32ac09d44364c4', range_limit=1)
wbtc_dai_burns_df = pool_burns_to_df(wbtc_dai_burns)
wbtc_dai_burns_df

100%|██████████| 1/1 [00:00<00:00,  2.02it/s]


,amount0,amount1,amountUSD,liquidity,timestamp
0,0.000020,0.198396,0.393071,2.000000e-08,2020-05-18 21:23:57
1,0.100134,940.023349,1879.119093,9.699443e-05,2020-05-29 21:32:50
2,0.164053,1560.514491,3113.745579,1.599430e-04,2020-06-01 12:07:18
3,0.110400,1095.423005,2144.658189,1.098884e-04,2020-06-11 15:32:31
4,0.205953,1835.102135,3700.348283,1.942387e-04,2020-06-15 14:15:28
...,...,...,...,...,...
133,0.080422,3671.205894,7354.563725,1.587508e-04,2021-09-11 18:06:20
134,6.832808,328088.100764,656524.081660,1.380124e-02,2021-10-02 17:14:22
135,0.307307,18703.775899,37362.639081,6.976669e-04,2021-10-17 00:10:29
136,0.029345,1694.250980,3393.591808,6.473561e-05,2021-11-20 18:17:44


### Writing swaps, mints, burns, reserves into pickle files

In [28]:
wbtc_dai_df.to_pickle(os.getcwd() + "\\pkl_stories\\wbtc_dai_swaps.pkl")
wbtc_dai_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\wbtc_dai_reserves.pkl")
wbtc_dai_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\wbtc_dai_mints.pkl")
wbtc_dai_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\wbtc_dai_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [29]:
wbtc_dai_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\wbtc_dai_swaps.pkl")
wbtc_dai_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\wbtc_dai_reserves.pkl")
wbtc_dai_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\wbtc_dai_mints.pkl")
wbtc_dai_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\wbtc_dai_burns.pkl")

# IXS/ETH pool information

## Getting IXS/ETH swaps, mints, burns, reserves data

In [30]:
ixs_weth_history = get_pool_v2_history(contract_id='0xc09bf2b1bc8725903c509e8caeef9190857215a8', range_limit=4)
all_ixs_weth_history, other_ixs_weth_history = filter_swaps(ixs_weth_history)
ixs_weth_df = pool_history_to_df(all_ixs_weth_history)
ixs_weth_df

100%|██████████| 4/4 [00:04<00:00,  1.03s/it]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,WETH,IXS,12.000000,211315.866941,49367.994152,2021-09-08 12:59:29,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x03a92ec2e6ceee63731cd49d99050d24a40ba25a,0x2df317ab076e29ca317edf45a6e484a3acef1674def4...
1,IXS,WETH,10781.086133,1.010521,3412.553806,2021-09-08 12:59:42,0x1d6e8bac6ea3730825bde4b005ed7b2b39a2932d,0x1d6e8bac6ea3730825bde4b005ed7b2b39a2932d,0x05314a2d9bb717a8b4d3b4349d698e9e8c5fd08a6421...
2,WETH,IXS,0.886100,10781.086133,3050.396683,2021-09-08 12:59:42,0x1d6e8bac6ea3730825bde4b005ed7b2b39a2932d,0x1d6e8bac6ea3730825bde4b005ed7b2b39a2932d,0x1383c605487ded022ad75231fcb5383841b0c4ef2241...
3,WETH,IXS,3.000000,33333.139648,10583.791945,2021-09-08 12:59:42,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x3d55017098970268414f4b9252fdbe18cd09dcc3,0xad4556f77687567af93ea9587d89be78bff951133808...
4,WETH,IXS,0.146153,1580.639685,498.586381,2021-09-08 12:59:58,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xdf33624ed4623a3dea8be15ce5a4623d26d0a246,0x4dc080fadc4b5c1abe6613250689a672b54ab06db779...
...,...,...,...,...,...,...,...,...,...
2048,WETH,IXS,1.868507,25000.000000,7983.186569,2021-11-22 14:20:49,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x358efdbf79a4f8171591f8718d848636b07c37f1,0xd8b8b2feb8ec382f5c2acb8edc737912a87688d681c5...
2049,IXS,WETH,11449.090000,0.859735,3607.511933,2021-11-23 10:21:57,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x5dffd52e4a4efb83a3208971027739887ba5bf77193a...
2050,IXS,WETH,25000.000000,1.823572,7688.284273,2021-11-23 14:33:02,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0x970860e889031addb52f7dd3a405edc57c1f7217b040...
2051,WETH,IXS,1.369158,18751.392476,5770.940827,2021-11-23 14:34:56,0x0d89631141a5642a2dd845305d10d9bfca8b6b82,0xabd69d0fac4b0851dafe100979df808eb7fb81a9,0xa497fa78cb62d62d25491091d0876f5ed1a60b1b2f7e...


In [31]:
ixs_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0xc09bf2b1bc8725903c509e8caeef9190857215a8')
ixs_weth_reserves_df = pool_reserves_to_df(ixs_weth_reserves_history)
ixs_weth_reserves_df

100%|██████████| 2/2 [00:00<00:00,  3.80it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,3.794422e+05,55.944001,390788.928776,3.245787e+06,425.302832,2021-09-08
1,3.880956e+05,54.966786,376226.981271,6.385440e+05,93.661413,2021-09-09
2,3.895395e+05,54.992914,352105.690608,5.129601e+05,85.994939,2021-09-10
3,3.251391e+05,66.178741,431450.660405,5.158716e+05,94.863760,2021-09-11
4,3.138946e+05,68.793197,472201.658618,3.769885e+05,81.732506,2021-09-12
...,...,...,...,...,...,...
73,1.209956e+06,98.094824,862454.304721,1.672137e+05,13.709443,2021-11-20
74,1.284852e+06,92.456066,797584.511803,3.727151e+05,26.932798,2021-11-21
75,1.247104e+06,94.797725,810045.759781,9.639416e+04,7.093675,2021-11-22
76,1.252266e+06,92.530748,780026.110262,5.520048e+04,4.052464,2021-11-23


In [32]:
ixs_weth_mints_history = get_pool_v2_mints(contract_id='0xc09bf2b1bc8725903c509e8caeef9190857215a8', range_limit=2)
ixs_weth_mints_df = pool_mints_to_df(ixs_weth_mints_history)
ixs_weth_mints_df

100%|██████████| 2/2 [00:01<00:00,  1.89it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,720000.000000,28.800000,98227.795887,4553.679831,2021-09-08 12:59:29
1,1360.163496,0.195028,1414.289538,15.913010,2021-09-16 13:03:24
2,14999.994987,2.150790,15598.796295,175.489987,2021-09-16 13:04:20
3,10461.268874,1.500000,10880.913058,122.389903,2021-09-16 13:06:30
4,3010.000000,0.412264,2993.141508,34.416269,2021-09-16 13:12:28
...,...,...,...,...,...
111,16166.492978,1.950000,17012.881531,171.196836,2021-11-01 14:01:52
112,1428.439848,0.146827,1315.281150,13.959495,2021-11-02 14:16:58
113,3293.122422,0.306428,2777.935907,30.615115,2021-11-03 17:13:47
114,1000.000216,0.075075,716.351795,8.343178,2021-11-09 18:30:44


In [33]:
ixs_weth_burns_history = get_pool_v2_burns(contract_id='0xc09bf2b1bc8725903c509e8caeef9190857215a8', range_limit=2)
ixs_weth_burns_df = pool_burns_to_df(ixs_weth_burns_history)
ixs_weth_burns_df.head(5)

100%|██████████| 2/2 [00:00<00:00,  3.67it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,635.707371,0.086897,630.742972,7.261432,2021-09-16 13:25:29
1,7.493118,0.000959,6.934917,0.082742,2021-09-16 16:44:04
2,3169.601280,0.405602,2929.932964,35.000000,2021-09-16 16:55:51
3,2160.670975,0.285767,2034.020295,24.254484,2021-09-16 18:41:38
4,644.837670,0.085285,609.778679,7.238587,2021-09-16 19:40:37


### Saving swaps, mints, burns, reserves into pickle files

In [34]:
ixs_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\ixs_weth_swaps.pkl")
ixs_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\ixs_weth_reserves.pkl")
ixs_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\ixs_weth_mints.pkl")
ixs_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\ixs_weth_burns.pkl")

### Loading swaps, mints, burns, reserves from pickle files

In [35]:
ixs_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\ixs_weth_swaps.pkl")
ixs_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\ixs_weth_reserves.pkl")
ixs_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\ixs_weth_mints.pkl")
ixs_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\ixs_weth_burns.pkl")

# SQUID/ETH pool 

It is important to mention that this pool was chosen considering that it became a part of a fraud scheme performed by the creators of the SQUID token and it will be interesting to go through this story, run simulation over it.

In [36]:
squid_weth_history = get_pool_v2_history(contract_id='0x40df4eb23d23bb7af5c6bb7bc9a185913d97e7b9', range_limit=4)
all_squid_weth_history, other_squid_weth_history = filter_swaps(squid_weth_history)
squid_weth_df = pool_history_to_df(all_squid_weth_history)
squid_weth_df

 50%|█████     | 2/4 [00:00<00:00,  2.37it/s]

list index out of range


 75%|███████▌  | 3/4 [00:01<00:00,  2.36it/s]

list index out of range


100%|██████████| 4/4 [00:01<00:00,  2.51it/s]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,SQUID,WETH,1.515152e+10,0.882363,2368.034156,2021-09-29 21:58:26,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x817a2564c2300f96b539332fde239bfa2f7a44a5a9c5...
1,SQUID,WETH,1.274336e+10,0.619296,1680.994807,2021-09-29 21:58:26,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x817a2564c2300f96b539332fde239bfa2f7a44a5a9c5...
2,SQUID,WETH,9.266101e+09,0.473672,1298.817313,2021-09-29 21:59:00,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xcda98c33dcdff5252700cac9921313c8d5e6b1569e6f...
3,SQUID,WETH,1.263748e+10,0.565413,1538.285490,2021-09-29 21:59:00,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xcda98c33dcdff5252700cac9921313c8d5e6b1569e6f...
4,SQUID,WETH,1.515152e+10,0.945236,2533.074357,2021-09-29 21:59:00,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xdda4b205d0b44eb3030010f6ac32ced6b62e6211e51f...
...,...,...,...,...,...,...,...,...,...
173,SQUID,WETH,1.017555e+10,0.042067,147.895471,2021-10-10 13:15:24,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xba307bef6a8f4609a82bbfd66623f310a7f21c3d2d82...
174,SQUID,WETH,7.732504e+08,0.003135,10.716093,2021-10-12 12:54:22,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x54049a82a84a87145125e6c8c147bf970489bfe25574...
175,SQUID,WETH,9.998849e+09,0.039789,134.914133,2021-10-12 12:54:22,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x54049a82a84a87145125e6c8c147bf970489bfe25574...
176,SQUID,WETH,8.645340e+08,0.003396,13.140648,2021-10-18 06:52:26,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2c1d79cb2425904c1be268687e275df742f8a352d100...


In [37]:
squid_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0x40df4eb23d23bb7af5c6bb7bc9a185913d97e7b9')
squid_weth_reserves_df = pool_reserves_to_df(squid_weth_reserves_history)
squid_weth_reserves_df

100%|██████████| 2/2 [00:00<00:00,  3.11it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,4.844201e+11,2.705857,15381.062606,1.976699e+12,38.460760,2021-09-29
1,5.625445e+11,2.336738,14192.247090,2.373009e+11,1.003119,2021-09-30
2,5.699927e+11,2.306294,15185.894105,7.448186e+09,0.030444,2021-10-02
3,5.686125e+11,2.311974,16624.768469,1.411990e+09,0.005680,2021-10-07
4,5.682726e+11,2.314200,16394.571671,2.366301e+10,0.097774,2021-10-10
5,5.790447e+11,2.271275,15513.444104,1.077210e+10,0.042925,2021-10-12
6,5.773171e+11,2.278172,17561.696928,1.767880e+09,0.006897,2021-10-17
7,5.900352e+11,2.229211,17237.288891,1.271807e+10,0.048961,2021-10-18
8,5.760329e+11,2.284211,18598.841009,1.432866e+10,0.055000,2021-10-20
9,5.638690e+11,2.334211,19218.093319,1.244144e+10,0.050000,2021-10-22


In [38]:
squid_weth_mints_history = get_pool_v2_mints(contract_id='0x40df4eb23d23bb7af5c6bb7bc9a185913d97e7b9', range_limit=2)
squid_weth_mints_df = pool_mints_to_df(squid_weth_mints_history)
squid_weth_mints_df

100%|██████████| 2/2 [00:00<00:00,  4.32it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,7.381000e+11,1.7,4788.810094,35.422733,2021-09-29 21:57:29


There are no burns in the pool

In [39]:
squid_weth_burns_history = get_pool_v2_burns(contract_id='0x40df4eb23d23bb7af5c6bb7bc9a185913d97e7b9', range_limit=1)
squid_weth_burns_df = pool_burns_to_df(squid_weth_burns_history)
squid_weth_burns_df

100%|██████████| 1/1 [00:00<00:00,  3.94it/s]

list index out of range


AttributeError: 'DataFrame' object has no attribute 'amount0'

### Writing swaps, mints and reserves into pickle files

In [40]:
squid_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\squid_weth_swaps.pkl")
squid_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\squid_weth_reserves.pkl")
squid_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\squid_weth_mints.pkl")

### Reading swaps, mints and reserves from pickle files

In [41]:
squid_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\squid_weth_swaps.pkl")
squid_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\squid_weth_reserves.pkl")
squid_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\squid_weth_mints.pkl")

# HKMT/USDT pool

In [42]:
hkmt_usdt_history = get_pool_v2_history(contract_id='0xf52f433b79d21023af94251958bed3b64a2b7930', range_limit=4)
all_hkmt_usdt_history, other_hkmt_usdt_history = filter_swaps(hkmt_usdt_history)
hkmt_usdt_df = pool_history_to_df(all_hkmt_usdt_history)
hkmt_usdt_df

 75%|███████▌  | 3/4 [00:01<00:00,  2.97it/s]

list index out of range
list index out of range


100%|██████████| 4/4 [00:01<00:00,  2.82it/s]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,USDT,HKMT,1388.137176,10725.294140,1377.513809,2020-09-01 09:06:15,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xb9ccfc18b941a2cc51a956aa6c83360eb9d58de7,0x9c87f160114aa1bdc22cf332b99df1212b3df326bbbd...
1,USDT,HKMT,2315.057158,17884.839162,2296.644225,2020-09-01 09:07:04,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xc5f3106f0a40b81b070a074bee1fdcd21452943b,0xd0096539f938f8d9da41874290e6f6467ff31c3200c0...
2,HKMT,USDT,10725.294140,1380.033860,1369.211622,2020-09-01 09:07:36,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xb9ccfc18b941a2cc51a956aa6c83360eb9d58de7,0x66143b4a40bc38c41b55c6934938936722628ad001e0...
3,USDT,HKMT,42.510850,328.419546,42.275994,2020-09-01 09:25:52,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xa260d88f951f53e38c99f43c8a6bf52df89334f7,0x4bd8bce026f1e5ccff9b72063b9991298f043afe07f3...
4,HKMT,USDT,17884.839162,2300.983230,2293.554386,2020-09-01 09:32:21,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852,0x79c6a7a64614a1004c4047c2a4e8dfa77adb68715cbd...
...,...,...,...,...,...,...,...,...,...
337,HKMT,USDT,29450.000000,3817.212732,3811.229793,2021-11-14 02:32:53,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x071e3ee0a8fd2aaefc95344aaab62bcb484b016c,0xfcc48b8045c56367bc6553091e92b93df6395b7fbead...
338,HKMT,USDT,281823.000000,36499.627099,36423.861132,2021-11-15 01:22:21,0x11111112542d85b3ef69ae05771c2dccff4faa26,0x74de5d4fcbf63e00296fd95d33236b9794016631,0x3d3852cbb75bea9ca1449aa04889590ae4c0ff04e650...
339,HKMT,USDT,30000.000000,3882.243192,3879.683977,2021-11-15 03:23:42,0x11111112542d85b3ef69ae05771c2dccff4faa26,0x4087d0e6e513f260de87408bee9334a5742cfdf4,0x14296130144edb9cd065726f97e50c5a18b8e5120dea...
340,HKMT,USDT,402277.856847,52000.000000,52079.153298,2021-11-16 03:40:54,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xd7b5c42c00a7e3fd3841db567b0c401044880b09,0x40d495a37ab4a43a1baa00c24b118fde011bb34f8ebc...


In [43]:
hkmt_usdt_reserves_history = get_pool_v2_reserves_history(contract_id='0xf52f433b79d21023af94251958bed3b64a2b7930')
hkmt_usdt_reserves_df = pool_reserves_to_df(hkmt_usdt_reserves_history)
hkmt_usdt_reserves_df

100%|██████████| 2/2 [00:00<00:00,  2.65it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,2.324374e+08,3.000841e+07,5.962596e+07,841180.034572,108718.139712,2020-09-01
1,2.324720e+08,3.000396e+07,5.998884e+07,39685.978673,5109.376059,2020-09-02
2,2.324620e+08,3.000526e+07,5.980636e+07,12136.201494,1570.298902,2020-09-03
3,2.324728e+08,3.000387e+07,5.974688e+07,11266.150769,1449.936710,2020-09-04
4,2.324795e+08,3.000300e+07,5.981387e+07,7698.807460,990.995574,2020-09-05
...,...,...,...,...,...,...
154,3.860615e+08,5.019442e+07,1.002105e+08,83900.000000,10878.034552,2021-11-12
155,3.860910e+08,5.019060e+07,1.002239e+08,29450.000000,3817.212732,2021-11-14
156,3.864028e+08,5.015022e+07,1.002343e+08,311823.000000,40381.870291,2021-11-15
157,3.868051e+08,5.009822e+07,1.003490e+08,402277.856847,52000.000000,2021-11-16


In [44]:
hkmt_usdt_mints_history = get_pool_v2_mints(contract_id='0xf52f433b79d21023af94251958bed3b64a2b7930', range_limit=2)
hkmt_usdt_mints_df = pool_mints_to_df(hkmt_usdt_mints_history)
hkmt_usdt_mints_df

100%|██████████| 2/2 [00:00<00:00,  3.56it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,2.325000e+08,3.000000e+07,2.980819e+07,8.351647e-05,2020-09-01 09:05:07
1,1.549714e+08,2.000000e+07,3.987585e+07,5.567218e-05,2020-09-07 06:32:58
2,1.435528e+02,1.852842e+01,3.681437e+01,5.157298e-11,2020-09-14 14:16:14
3,1.500000e+02,1.936058e+01,3.852151e+01,5.388924e-11,2020-09-14 17:06:08
4,5.760610e+02,7.435276e+01,1.478349e+02,2.069570e-10,2020-09-14 18:46:57
5,3.089770e+02,3.988003e+01,7.934419e+01,1.110039e-10,2020-09-15 01:41:11
6,2.897847e+01,3.740286e+00,7.427972e+00,1.041088e-11,2020-09-15 04:43:57
7,2.915278e+02,3.762785e+01,7.489703e+01,1.047350e-10,2020-09-15 12:42:05
8,1.000000e+01,1.290712e+00,2.567839e+00,3.592625e-12,2020-09-15 12:50:36
9,2.905769e+03,3.750568e+02,7.477305e+02,1.043942e-09,2020-09-15 12:57:14


In [45]:
hkmt_usdt_burns_history = get_pool_v2_burns(contract_id='0xf52f433b79d21023af94251958bed3b64a2b7930', range_limit=2)
hkmt_usdt_burns_df = pool_burns_to_df(hkmt_usdt_burns_history)
hkmt_usdt_burns_df

100%|██████████| 2/2 [00:00<00:00,  3.95it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,5665.201219,731.338390,1460.247116,2.035467e-09,2020-09-16 16:40:02
1,5684.618907,733.929268,1466.409881,2.042560e-09,2020-09-16 18:57:59
2,27801.582229,3589.298650,7157.725005,9.989337e-09,2020-09-16 19:28:07
3,143.544388,18.529515,36.941449,5.157298e-11,2020-09-16 20:15:33
4,14203.219302,1833.429928,3663.026267,5.102965e-09,2020-09-17 00:21:13
5,576.034022,74.356343,147.592964,2.069570e-10,2020-09-18 05:26:54
6,3846.609344,496.516072,987.800659,1.381983e-09,2020-09-20 13:52:07
7,291.543450,37.625893,75.102954,1.047350e-10,2020-10-06 12:53:08
8,25.564644,3.280733,6.558273,9.157992e-12,2020-12-06 09:20:37
9,78.414536,9.940579,19.897154,2.791746e-11,2021-05-26 00:50:16


### Writing swaps, mints, burns, reserves

In [46]:
hkmt_usdt_df.to_pickle(os.getcwd() + "\\pkl_stories\\hkmt_usdt_swaps.pkl")
hkmt_usdt_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\hkmt_usdt_reserves.pkl")
hkmt_usdt_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\hkmt_usdt_mints.pkl")
hkmt_usdt_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\hkmt_usdt_burns.pkl")

### Loading swaps, mints, burns, reserves

In [47]:
hkmt_usdt_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\hkmt_usdt_swaps.pkl")
hkmt_usdt_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\hkmt_usdt_reserves.pkl")
hkmt_usdt_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\hkmt_usdt_mints.pkl")
hkmt_usdt_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\hkmt_usdt_burns.pkl")

# FEI/ETH pool

## Getting FEI/ETH swaps, mints, burns, reserves

In [49]:
fei_weth_history = get_pool_v2_history(contract_id='0x94b0a3d511b6ecdb17ebf877278ab030acb0a878', range_limit=51)
all_fei_weth_history, other_fei_weth_history = filter_swaps(fei_weth_history)
fei_weth_df = pool_history_to_df(all_fei_weth_history)
fei_weth_df

100%|██████████| 51/51 [01:25<00:00,  1.68s/it]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,FEI,WETH,1.945271e+06,928.783242,1.916057e+06,2021-04-03 19:00:36,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x3347f819f2bf7e7c54f59b97fecb84399ff479f93d2f...
1,FEI,WETH,9.518713e+05,455.470626,9.396247e+05,2021-04-03 19:00:36,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x59e7772688fe9966abfeba79408ca9aedbf316bf4761...
2,FEI,WETH,7.580513e+06,3650.791147,7.531493e+06,2021-04-03 19:00:36,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2ebd449e7b10b20135b6c1134b9e40c84f9fffde,0x641b95d1f76a4a9e0101df94aab04aee23f95bca349f...
3,FEI,WETH,9.905835e+05,471.915956,9.734749e+05,2021-04-03 19:01:19,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2fd4a218b8733ebe9e3ebf950e3aae1e0981b3b336de...
4,WETH,FEI,2.000000e-01,417.609981,4.130432e+02,2021-04-03 19:01:19,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xc3ec7b35180768cf73d2f5ff9ec82485106d5d7c,0x3dd3e1b61fbe1dcfc3c5a60ceed774b914704df32ba7...
...,...,...,...,...,...,...,...,...,...
49958,WETH,FEI,6.105187e+01,260172.839630,2.600585e+05,2021-11-24 13:50:29,0x51399b32cd0186bb32230e24167489f3b2f47870,0x51399b32cd0186bb32230e24167489f3b2f47870,0xd0221cf2e2dd994d704ebbdb5b1e1c69fd74880174a0...
49959,WETH,FEI,1.796507e+01,76382.251115,7.647840e+04,2021-11-24 13:51:13,0x220bda5c8994804ac96ebe4df184d25e5c2196d4,0x220bda5c8994804ac96ebe4df184d25e5c2196d4,0xedd8593b6ffce172d811e41e5dd5751c722c3d725bc2...
49960,WETH,FEI,6.119620e+01,259591.196503,2.601098e+05,2021-11-24 13:52:19,0x51399b32cd0186bb32230e24167489f3b2f47870,0x51399b32cd0186bb32230e24167489f3b2f47870,0xd382b2a2a6d6cd26af7779a5dcc208cd7cb37bf57ef6...
49961,FEI,WETH,2.000000e+02,0.046949,1.988372e+02,2021-11-24 13:57:46,0x220bda5c8994804ac96ebe4df184d25e5c2196d4,0x220bda5c8994804ac96ebe4df184d25e5c2196d4,0x50345d6ce455bd80b19bb5379f32fc505712ae268622...


In [50]:
fei_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0x94b0a3d511b6ecdb17ebf877278ab030acb0a878')
fei_weth_reserves_df = pool_reserves_to_df(fei_weth_reserves_history)
fei_weth_reserves_df

100%|██████████| 2/2 [00:00<00:00,  3.13it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,1.337230e+09,630522.677036,2.537871e+09,5.830436e+07,27452.763852,2021-04-03
1,1.341220e+09,628533.678068,2.604474e+09,6.647175e+07,31306.853434,2021-04-04
2,1.337334e+09,610770.369281,2.561514e+09,8.717716e+07,40829.752667,2021-04-05
3,1.323221e+09,598300.386209,2.531489e+09,5.256349e+07,24003.259833,2021-04-06
4,1.192320e+09,490330.663557,1.925378e+09,2.657121e+08,122248.657720,2021-04-07
...,...,...,...,...,...,...
231,1.489893e+08,33726.798670,2.971007e+08,7.857096e+06,1822.865090,2021-11-20
232,1.465835e+08,34284.616392,2.937705e+08,6.154276e+06,1414.759309,2021-11-21
233,1.435460e+08,35019.487375,2.869859e+08,1.292556e+07,3094.474835,2021-11-22
234,1.477743e+08,34026.568066,2.956214e+08,1.295598e+07,3085.020510,2021-11-23


In [51]:
fei_weth_mints_history = get_pool_v2_mints(contract_id='0x94b0a3d511b6ecdb17ebf877278ab030acb0a878', range_limit=2)
fei_weth_mints_df = pool_mints_to_df(fei_weth_mints_history)
fei_weth_mints_df

100%|██████████| 2/2 [00:01<00:00,  1.64it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,1.315771e+09,639235.592414,1.318727e+09,2.900151e+07,2021-04-03 18:59:46
1,7.688361e+03,3.994820,1.578041e+04,1.666953e+02,2021-04-03 19:10:31
2,1.255904e+05,64.904372,2.567463e+05,2.716398e+03,2021-04-03 19:19:25
3,2.073553e+01,0.009700,4.033823e+01,4.484614e-01,2021-04-03 19:19:42
4,6.508717e+02,0.333262,1.327281e+03,1.408814e+01,2021-04-03 19:23:25
...,...,...,...,...,...
596,1.324539e+08,37577.176787,2.643709e+08,2.151832e+06,2021-10-10 19:04:42
597,4.278637e+04,12.138499,8.539863e+04,6.951029e+02,2021-10-10 19:04:56
598,3.940595e+02,0.102231,7.900858e+02,6.116372e+00,2021-10-18 05:47:04
599,6.481661e+01,0.014953,1.293932e+02,9.460833e-01,2021-11-16 07:33:05


In [52]:
fei_weth_burns_history = get_pool_v2_burns(contract_id='0x94b0a3d511b6ecdb17ebf877278ab030acb0a878', range_limit=2)
fei_weth_burns_df = pool_burns_to_df(fei_weth_burns_history)
fei_weth_burns_df

100%|██████████| 2/2 [00:01<00:00,  1.68it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,6.517701e+02,0.304577,1.268090e+03,1.408814e+01,2021-04-03 19:56:32
1,2.201983e+04,10.277614,4.288282e+04,4.756753e+02,2021-04-03 20:00:59
2,3.826946e+03,1.778466,7.337225e+03,8.249027e+01,2021-04-03 20:49:31
3,4.349987e+03,2.034685,8.270638e+03,9.406615e+01,2021-04-03 22:28:41
4,3.641036e-02,0.000017,6.908305e-02,7.873546e-04,2021-04-03 22:29:18
...,...,...,...,...,...
520,1.316554e+08,37276.459551,2.618904e+08,2.136850e+06,2021-10-10 01:33:20
521,1.324967e+08,37589.319208,2.639461e+08,2.152527e+06,2021-10-10 18:48:10
522,1.039316e+03,0.296067,2.083135e+03,1.691675e+01,2021-10-11 22:19:02
523,4.287733e+02,0.094292,8.565958e+02,6.116372e+00,2021-11-05 08:06:51


### Writing swaps, mints, burns, reserves to pickle files

In [53]:
fei_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\fei_weth_swaps.pkl")
fei_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\fei_weth_reserves.pkl")
fei_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\fei_weth_mints.pkl")
fei_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\fei_weth_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [54]:
fei_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\fei_weth_swaps.pkl")
fei_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\fei_weth_reserves.pkl")
fei_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\fei_weth_mints.pkl")
fei_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\fei_weth_burns.pkl")

# ETH/USDC pool

In [ ]:
weth_usdc_history = get_pool_v2_history(contract_id='0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc', range_limit=2930)
all_weth_usdc_history, other_weth_usdc_history = filter_swaps(weth_usdc_history)
weth_usdc_df = pool_history_to_df(all_weth_usdc_history)
weth_usdc_df

In [56]:
weth_usdc_reserves_history = get_pool_v2_reserves_history(contract_id='0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc')
weth_usdc_reserves_df = pool_reserves_to_df(weth_usdc_reserves_history)
weth_usdc_reserves_df

100%|██████████| 2/2 [00:05<00:00,  2.60s/it]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,9.890000e-01,0.004909,9.890000e-01,1.100000e-02,0.000054,2020-05-05
1,9.886890e-01,0.004910,9.886890e-01,1.689000e-03,0.000008,2020-05-06
2,9.886900e-01,0.004910,9.886900e-01,0.000000e+00,0.000000,2020-05-07
3,6.574471e+00,0.030998,6.574471e+00,0.000000e+00,0.000000,2020-05-08
4,6.562244e+00,0.031061,6.562244e+00,3.975130e-01,0.001937,2020-05-11
...,...,...,...,...,...,...
562,1.188560e+08,26972.513230,2.376582e+08,2.624093e+07,6081.945816,2021-11-20
563,1.171459e+08,27417.693942,2.343634e+08,2.887490e+07,6631.914885,2021-11-21
564,1.117861e+08,27301.119535,2.237727e+08,4.594772e+07,11064.868711,2021-11-22
565,1.149209e+08,26433.346559,2.298133e+08,3.883966e+07,9202.194266,2021-11-23


In [ ]:
weth_usdc_mints_history = get_pool_v2_mints(contract_id='0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc', range_limit=34)
weth_usdc_mints_df = pool_mints_to_df(weth_usdc_mints_history)
weth_usdc_mints_df

In [58]:
weth_usdc_burns_history = get_pool_v2_burns(contract_id='0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc', range_limit=24)
weth_usdc_burns_df = pool_burns_to_df(weth_usdc_burns_history)
weth_usdc_burns_df

100%|██████████| 24/24 [00:35<00:00,  1.49s/it]


,amount0,amount1,amountUSD,liquidity,timestamp
0,0.003262,0.000016,0.003263,2.299449e-10,2020-05-07 15:47:25
1,0.014189,0.000070,0.014190,9.999739e-10,2020-05-08 02:43:59
2,0.781284,0.006539,0.781285,6.960485e-08,2020-05-12 20:18:50
3,0.995309,0.004114,0.836172,6.217433e-08,2020-05-13 23:48:21
4,1.097105,0.005132,1.047697,7.289922e-08,2020-05-14 14:24:41
...,...,...,...,...,...
23363,135763.610240,31.582256,271235.041449,1.150000e-03,2021-11-24 10:17:19
23364,3244.371870,0.755808,6483.561276,2.750132e-05,2021-11-24 10:36:26
23365,29556.761190,6.882933,59073.445800,2.504936e-04,2021-11-24 10:45:53
23366,2223.947507,0.518301,4454.537870,1.885461e-05,2021-11-24 13:26:04


### Writing swaps, mints, burns, reserves into pickle files

In [61]:
weth_usdc_df.to_pickle(os.getcwd() + "\\pkl_stories\\weth_usdc_swaps.pkl")
weth_usdc_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\weth_usdc_reserves.pkl")
weth_usdc_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\weth_usdc_mints.pkl")
weth_usdc_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\weth_usdc_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [62]:
weth_usdc_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\weth_usdc_swaps.pkl")
weth_usdc_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\weth_usdc_reserves.pkl")
weth_usdc_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\weth_usdc_mints.pkl")
weth_usdc_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\weth_usdc_burns.pkl")

# AXS/ETH v2 pool - NFT

In [64]:
axs_weth_v2_history = get_pool_v2_history(contract_id='0xa040832b1e5026da6dc843e6773a9e4d02fff9b1', range_limit=5)
all_axs_weth_v2_history, other_axs_weth_v2_history = filter_swaps(axs_weth_v2_history)
axs_weth_v2_df = pool_history_to_df(all_axs_weth_v2_history)
axs_weth_v2_df

100%|██████████| 5/5 [00:05<00:00,  1.10s/it]


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,WETH,AXS,1.000000,271.895117,3265.028270,2021-05-03 01:42:22,0x27f9adb26d532a41d97e00206114e429ad58c679,0x27f9adb26d532a41d97e00206114e429ad58c679,0x4bd0e1ac628e46befe15c93c7f5125146dbcb3d39dec...
1,WETH,AXS,0.400000,87.762331,1238.364115,2021-05-03 01:42:28,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x9108048f5ef19c64d00adda718906990b53d02cd,0x0439383f710521f46e1af7355efc9462c0b002dec9c5...
2,WETH,AXS,0.190000,0.048255,572.499245,2021-05-03 01:42:48,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xa097e13bef6a6985008924af02f3afdb8814391d,0x25d4e2eb74505fdecf75fcb7bb872f5c00d81e3fcf7c...
3,WETH,AXS,0.010000,0.002537,30.131539,2021-05-03 01:42:48,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xc496d84215d5018f6f53e7f6f12e45c9b5e8e8a9,0x25d4e2eb74505fdecf75fcb7bb872f5c00d81e3fcf7c...
4,WETH,AXS,0.300000,59.387987,921.135675,2021-05-03 01:42:48,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xbf5d676f2d3c1a5abd27c4b11a95541d79575d5d,0x51b79e91294e4b3a894d8b2fc239a6a01dbde7b92734...
...,...,...,...,...,...,...,...,...,...
4583,AXS,WETH,1.100000,0.034694,150.985964,2021-11-23 22:20:45,0x86d3579b043585a97532514016dcf0c2d6c4b6a1,0xbb2b8038a1640196fbe3e38816f3e67cba72d940,0xd955764bf91c159a69073dbee1f2c6b2009313d31295...
4584,AXS,WETH,5.967619,0.182926,786.438300,2021-11-23 23:45:07,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x0612f7b15779ca6ddbe927f92d6c4144fbae1485,0x19f7f813f1a2632439d675d2dddc576d313ede9a5ee6...
4585,AXS,WETH,1.000000,0.029805,129.691173,2021-11-24 00:14:44,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xd2b0964ca46a4f3401ef65bbf9c616ed2f43f01b,0x79cc9376be9d15762f36800b5e4e379e4e3373335a29...
4586,WETH,AXS,0.232592,7.554260,1000.997880,2021-11-24 04:04:25,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0xe66b31678d6c16e9ebf358268a790b763c133750,0xf36bde9c3aa6ad3eb31cde7757b8296b79a14398de77...


In [65]:
axs_weth_v2_reserves_history = get_pool_v2_reserves_history(contract_id='0xa040832b1e5026da6dc843e6773a9e4d02fff9b1')
axs_weth_v2_reserves_df = pool_reserves_to_df(axs_weth_v2_reserves_history)
axs_weth_v2_reserves_df

100%|██████████| 2/2 [00:00<00:00,  3.02it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,2235.318021,6.589013,43248.099360,5631.549029,412.261553,2021-05-03
1,2468.971504,5.971757,39933.495620,876.346517,2.191252,2021-05-04
2,2407.284859,6.133823,42093.892594,1233.886645,3.094878,2021-05-05
3,2603.981326,5.557229,38883.103492,1681.480184,4.037121,2021-05-06
4,2148.038360,5.171886,35978.200367,2004.968493,4.749660,2021-05-07
...,...,...,...,...,...,...
184,252.442218,7.452112,65834.588230,98.465778,3.006264,2021-11-20
185,246.162280,7.645403,66640.534300,34.488264,1.085657,2021-11-21
186,243.314960,7.744437,63016.184730,102.397294,3.219262,2021-11-22
187,250.674402,7.523598,65373.214397,72.124948,2.255345,2021-11-23


In [66]:
axs_weth_v2_mints_history = get_pool_v2_mints(contract_id='0xa040832b1e5026da6dc843e6773a9e4d02fff9b1', range_limit=2)
axs_weth_v2_mints_df = pool_mints_to_df(axs_weth_v2_mints_history)
axs_weth_v2_mints_df

100%|██████████| 2/2 [00:00<00:00,  2.48it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,1866.449528,5.847000,17615.215682,1.044659e+02,2021-05-03 01:42:22
1,25.000000,0.071740,474.404409,1.332144e+00,2021-05-03 18:14:10
2,280.000000,0.803485,5298.366107,1.492001e+01,2021-05-03 18:31:59
3,50.000000,0.096979,380.827232,2.155080e+00,2021-05-15 09:01:25
4,3.982368,0.010916,41.793753,2.038269e-01,2021-05-16 02:37:43
...,...,...,...,...,...
67,1.000000,0.030401,261.888487,2.853140e-08,2021-11-16 07:40:53
68,43.782246,1.390685,11779.923177,1.276715e-06,2021-11-17 18:20:19
69,46.772919,1.425133,11601.914379,1.335378e-06,2021-11-18 21:13:42
70,20.006538,0.620373,5198.634382,5.754050e-07,2021-11-22 10:47:03


In [67]:
axs_weth_v2_burns_history = get_pool_v2_burns(contract_id='0xa040832b1e5026da6dc843e6773a9e4d02fff9b1', range_limit=2)
axs_weth_v2_burns_df = pool_burns_to_df(axs_weth_v2_burns_history)
axs_weth_v2_burns_df

100%|██████████| 2/2 [00:00<00:00,  3.98it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,26.456386,0.067974,468.479353,1.332144e+00,2021-05-06 05:26:36
1,326.295177,0.692936,4782.409399,1.492001e+01,2021-05-07 07:54:27
2,2423.697774,4.615325,34391.396160,1.044659e+02,2021-05-13 11:20:29
3,48.409812,0.103813,348.682796,2.175059e+00,2021-05-18 00:31:52
4,151.944075,0.271016,1455.393578,5.617203e+00,2021-06-02 10:25:06
5,608.541769,1.030020,5809.629691,2.183182e+01,2021-06-03 06:40:18
6,797.524914,1.410775,8139.684065,2.923736e+01,2021-06-03 10:27:19
7,440.578025,0.781316,4386.521501,1.612396e+01,2021-06-03 20:55:08
8,461.742241,0.748523,3927.824987,1.596272e+01,2021-06-05 20:19:51
9,94.302815,0.151797,751.189716,3.240915e+00,2021-06-09 00:28:03


### Writing swaps, mints, burns, reserves into pickle files

In [68]:
axs_weth_v2_df.to_pickle(os.getcwd() + "\\pkl_stories\\axs_weth_v2_swaps.pkl")
axs_weth_v2_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\axs_weth_v2_reserves.pkl")
axs_weth_v2_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\axs_weth_v2_mints.pkl")
axs_weth_v2_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\axs_weth_v2_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [69]:
axs_weth_v2_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\axs_weth_v2_swaps.pkl")
axs_weth_v2_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\axs_weth_v2_reserves.pkl")
axs_weth_v2_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\axs_weth_v2_mints.pkl")
axs_weth_v2_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\axs_weth_v2_burns.pkl")

# MANA/ETH pool (Decentraland/Ethereum) - NFT

In [70]:
mana_weth_history = get_pool_v2_history(contract_id='0x11b1f53204d03e5529f09eb3091939e4fd8c9cf3', range_limit=72)
all_mana_weth_history, other_mana_weth_history = filter_swaps(mana_weth_history)
mana_weth_df = pool_history_to_df(all_mana_weth_history)
mana_weth_df

100%|██████████| 72/72 [01:40<00:00,  1.40s/it]


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,WETH,MANA,0.050000,275.955186,5.317078,2020-05-18 20:24:15,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0x7440779a961306dd7d8f27acb67ab6496aa3570c,0x3fc44e5ee6779bcfaef10ce8a61f93e05aedfebff8d9...
1,MANA,WETH,800.000000,0.140614,29.162929,2020-05-19 00:25:27,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0x139665acff25e60636028420a2dff6b8e009cdfa9701...
2,WETH,MANA,0.005000,29.943369,1.063490,2020-05-19 01:29:18,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0xa53496e93e8183b62d80b3081ca6f8bd2cb9ada0,0xc0e676a272ce5cd33d5b9c36024f6d2e43c5429e86c7...
3,WETH,MANA,0.040000,235.611527,8.510489,2020-05-19 01:34:01,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0xe3d4b572c452138bf62cd0566ecea482f49b6424,0x395c4738a2d0991e06b58c18e69f614abf05402f9100...
4,WETH,MANA,0.033072,190.000000,6.980768,2020-05-19 06:44:23,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0x196a3dc8446920cef0f0d1f6bf7ba5b40702c79f,0xc5a68b1d08516ee37e0b648030dd88136db3c78fa62b...
...,...,...,...,...,...,...,...,...,...
71994,WETH,MANA,0.028746,29.451902,122.516521,2021-11-19 21:26:45,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0x74de5d4fcbf63e00296fd95d33236b9794016631,0x3ed6624e690b6fd4ea198f39646f5d4bb7d249be98b1...
71995,WETH,MANA,3.144065,3212.154135,13401.179611,2021-11-19 21:28:22,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0x95b47fa438762af1856a2d7dd68ef80af90c4ea9,0xa3b1487d38dc9889cb9f86f0b780b82c637f47336ed6...
71996,WETH,MANA,0.171873,175.075738,733.371124,2021-11-19 21:38:21,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x666e8e22895fae5e3f8fa42dedcb407e60859ccb,0xbfface16a7a11a1b20dd6b3eca3fe1233d440f281bab...
71997,WETH,MANA,1.936086,1968.473991,8256.181204,2021-11-19 21:46:53,0x6f8fa2495bd93c2a8f3984f1b1bd761bc1689eb4,0x6f8fa2495bd93c2a8f3984f1b1bd761bc1689eb4,0x0b11b300d032f33bdf00735aa43f8b8726462769af49...


In [71]:
mana_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0x11b1f53204d03e5529f09eb3091939e4fd8c9cf3')
mana_weth_reserves_df = pool_reserves_to_df(mana_weth_reserves_history)
mana_weth_reserves_df

100%|██████████| 2/2 [00:01<00:00,  1.66it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,1.313154e+04,2.455654,5.260017e+02,2.759552e+02,0.050000,2020-05-18
1,1.995563e+04,3.402474,1.456969e+03,1.545187e+03,0.268593,2020-05-19
2,2.320825e+04,4.142890,1.747858e+03,5.835603e+03,0.971594,2020-05-20
3,2.287628e+04,4.203910,1.674104e+03,1.630945e+03,0.304859,2020-05-21
4,4.713547e+04,9.083270,3.766017e+03,1.150966e+04,2.419877,2020-05-22
...,...,...,...,...,...,...
551,1.189052e+06,1088.299840,9.585300e+06,5.698641e+05,534.498749,2021-11-20
552,1.229540e+06,1053.815068,9.111223e+06,5.257195e+05,455.045753,2021-11-21
553,1.140786e+06,1140.525657,9.348950e+06,7.929923e+05,760.154427,2021-11-22
554,1.192215e+06,1096.862512,9.517764e+06,1.086525e+06,1066.868370,2021-11-23


In [72]:
mana_weth_mints_history = get_pool_v2_mints(contract_id='0x11b1f53204d03e5529f09eb3091939e4fd8c9cf3', range_limit=2)
mana_weth_mints_df = pool_mints_to_df(mana_weth_mints_history)
mana_weth_mints_df

100%|██████████| 2/2 [00:01<00:00,  1.32it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,8138.778341,1.420382,301.443838,107.518245,2020-05-18 19:40:36
1,197.637271,0.036959,7.921571,2.702542,2020-05-18 22:50:04
2,5071.078019,0.948313,203.128934,69.343211,2020-05-18 23:25:52
3,2238.013091,0.371897,159.418133,28.845815,2020-05-19 00:35:15
4,2810.821822,0.482834,202.898546,36.833688,2020-05-19 06:34:29
...,...,...,...,...,...
951,1992.997484,1.957821,16690.268917,37.815463,2021-11-19 21:49:12
952,98.868176,0.093216,798.905627,1.837127,2021-11-20 04:56:39
953,1483.217000,1.340178,11832.120197,26.969254,2021-11-20 23:21:05
954,1283.997884,1.155872,9786.653247,23.286858,2021-11-22 00:51:51


In [73]:
mana_weth_burns_history = get_pool_v2_burns(contract_id='0x11b1f53204d03e5529f09eb3091939e4fd8c9cf3', range_limit=2)
mana_weth_burns_df = pool_burns_to_df(mana_weth_burns_history)
mana_weth_burns_df

100%|██████████| 2/2 [00:01<00:00,  1.96it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,1119.607426,0.205747,81.280926,15.167027,2020-05-22 04:17:12
1,460.941138,0.085813,35.486995,6.281675,2020-05-23 19:44:19
2,2774.958609,0.466815,222.206766,35.891576,2020-05-31 05:19:22
3,24058.809930,4.143955,1999.222922,314.514600,2020-06-05 18:52:05
4,3003.841988,0.507565,240.105825,38.878333,2020-06-07 19:04:19
...,...,...,...,...,...
673,1602.694942,0.871143,8429.249388,23.017443,2021-11-10 14:18:22
674,80.266123,0.056017,516.742606,1.303296,2021-11-12 11:53:00
675,6233.196161,4.960454,45931.432289,107.868468,2021-11-13 04:19:26
676,737.471305,0.519906,4793.284037,11.988969,2021-11-14 11:29:56


### Writing swaps, mints, burns, reserves into pickle files

In [74]:
mana_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\mana_weth_swaps.pkl")
mana_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\mana_weth_reserves.pkl")
mana_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\mana_weth_mints.pkl")
mana_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\mana_weth_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [75]:
mana_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\mana_weth_swaps.pkl")
mana_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\mana_weth_reserves.pkl")
mana_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\mana_weth_mints.pkl")
mana_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\mana_weth_burns.pkl")

# ETH/ENJ pool - NFT

In [76]:
enj_eth_history = get_pool_v2_history(contract_id='0xe56c60b5f9f7b5fc70de0eb79c6ee7d00efa2625', range_limit=76)
all_enj_eth_history, other_enj_eth_history = filter_swaps(enj_eth_history)
enj_eth_df = pool_history_to_df(all_enj_eth_history)
enj_eth_df

100%|██████████| 76/76 [01:46<00:00,  1.40s/it]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,WETH,ENJ,0.250000,329.243700,53.924783,2020-05-19 00:27:03,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0x52cc3da7d6907b25fcd4e2c3b83f7edc5b31c949,0x70eb748663e95bcbfa16a060822a419c501198aeeaef...
1,WETH,ENJ,0.003249,4.206203,0.690529,2020-05-19 01:25:30,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0x1880b9b34fc7f59cb43dd53a6b5c6de9fedfd271,0x9bb753a638997e639952f3fc859e523877b35e0021e5...
2,WETH,ENJ,0.050000,64.528525,10.633417,2020-05-19 01:48:49,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0x537158466a8ca115157f0b291a984efa8e528876,0xad45c006d28353fc5e472a850237dadea38688234e7d...
3,WETH,ENJ,0.050000,64.156216,10.632978,2020-05-19 01:52:46,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0x0b2387b557e4f1ce550047cf1639249cd9db886c,0xc6d101a2253e47f58f62eb809cfca170542695561b6e...
4,WETH,ENJ,0.010000,12.786805,2.124114,2020-05-19 01:56:57,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0x672b8144fe28a33bfe9678e880ff2e78b4c82705,0xd7de4379d9a6c45e84d92da6ebade9de48da8e531c94...
...,...,...,...,...,...,...,...,...,...
74871,ENJ,WETH,2304.294278,2.326421,9906.938755,2021-11-24 16:02:08,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0xcbc8beb677625355510d257d15d30d972ce10c3b5b73...
74872,ENJ,WETH,841.767937,0.838110,3562.650658,2021-11-24 16:06:42,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x0cecc726d097c756c62272f1180ebfff29cb57f9,0x786495220790e0f4e3c22ac79fc9f2c347a2e12d7a6e...
74873,WETH,ENJ,2.697346,2671.158233,11453.855577,2021-11-24 16:07:23,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x4a137fd5e7a256ef08a7de531a17d0be0cc7b6b6,0x40fbfa1adfcbe84615fa540ec3cefa5dd13cde300ab7...
74874,WETH,ENJ,2.285582,2214.713838,9703.431273,2021-11-24 16:18:05,0x55ebdab1d9c3ab7220f388e7f7349e0af34cccd1,0x55ebdab1d9c3ab7220f388e7f7349e0af34cccd1,0x28efd6652f23a24127dcb6d446b4914eb882110cc0b7...


In [77]:
enj_eth_reserves_history = get_pool_v2_reserves_history(contract_id='0xe56c60b5f9f7b5fc70de0eb79c6ee7d00efa2625')
enj_eth_reserves_df = pool_reserves_to_df(enj_eth_reserves_history)
enj_eth_reserves_df

100%|██████████| 2/2 [00:01<00:00,  1.93it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,14.596187,19609.896031,6.256915e+03,0.000000,0.000000,2020-05-18
1,35.608058,45643.923958,1.503812e+04,4.334663,5636.075761,2020-05-19
2,44.083044,55306.165361,1.852330e+04,6.154577,8060.401355,2020-05-20
3,68.513132,76046.446474,2.728367e+04,39.970996,43953.282808,2020-05-21
4,70.230793,82183.521342,2.917812e+04,15.703982,18107.270743,2020-05-22
...,...,...,...,...,...,...
551,205.914460,249497.845790,1.813307e+06,126.858768,146488.397744,2021-11-20
552,200.857905,255877.179741,1.749590e+06,75.037519,91391.167819,2021-11-21
553,211.835049,243126.646149,1.737964e+06,149.054250,175787.658989,2021-11-22
554,214.183717,241353.778074,1.860132e+06,290.795102,295654.897573,2021-11-23


In [78]:
enj_eth_mints_history = get_pool_v2_mints(contract_id='0xe56c60b5f9f7b5fc70de0eb79c6ee7d00efa2625', range_limit=2)
enj_eth_mints_df = pool_mints_to_df(enj_eth_mints_history)
enj_eth_mints_df

100%|██████████| 2/2 [00:01<00:00,  1.40it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,12.826616,17232.487792,2738.593201,470.143074,2020-05-18 21:58:29
1,1.569474,2108.580074,672.783099,57.527058,2020-05-18 22:48:38
2,0.200096,268.828166,85.774806,7.334269,2020-05-18 22:49:38
3,2.356681,3060.607620,1002.815881,84.926512,2020-05-19 01:17:18
4,0.240934,315.332743,101.882671,8.715742,2020-05-19 13:20:41
...,...,...,...,...,...
947,0.756470,1152.000000,6880.642140,19.464956,2021-11-04 14:45:38
948,0.123547,178.909394,1029.580273,3.047213,2021-11-17 06:54:47
949,0.294524,340.200831,2490.228835,6.470337,2021-11-20 15:55:12
950,0.499812,526.395460,4240.486182,10.410603,2021-11-24 03:47:27


In [79]:
enj_eth_burns_history = get_pool_v2_burns(contract_id='0xe56c60b5f9f7b5fc70de0eb79c6ee7d00efa2625', range_limit=2)
enj_eth_burns_df = pool_burns_to_df(enj_eth_burns_history)
enj_eth_burns_df

100%|██████████| 2/2 [00:01<00:00,  1.76it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,0.324013,359.640139,128.844953,10.779454,2020-05-22 02:34:24
1,0.297889,354.773011,124.207311,10.262332,2020-05-22 18:42:33
2,4.020403,4879.430957,1685.933227,139.747002,2020-05-24 09:55:27
3,3.921471,4759.360616,1644.446714,136.308185,2020-05-24 09:56:27
4,11.301433,13591.069990,4615.327351,390.992199,2020-05-24 18:34:17
...,...,...,...,...,...
737,0.397470,609.870037,3602.440711,10.180784,2021-11-14 18:46:24
738,0.129985,190.482613,1105.859826,3.225924,2021-11-16 21:45:07
739,0.846058,1236.481371,7197.791438,20.955602,2021-11-18 06:07:06
740,0.071368,87.999242,620.633120,1.618900,2021-11-21 08:55:58


### Writing swaps, mints, burns, reserves into pickle files

In [80]:
enj_eth_df.to_pickle(os.getcwd() + "\\pkl_stories\\enj_eth_swaps.pkl")
enj_eth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\enj_eth_reserves.pkl")
enj_eth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\enj_eth_mints.pkl")
enj_eth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\enj_eth_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [81]:
enj_eth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\enj_eth_swaps.pkl")
enj_eth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\enj_eth_reserves.pkl")
enj_eth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\enj_eth_mints.pkl")
enj_eth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\enj_eth_burns.pkl")

# SAND/ETH pool - NFT

In [82]:
sand_weth_history = get_pool_v2_history(contract_id='0x3dd49f67e9d5bc4c5e6634b3f70bfd9dc1b6bd74', range_limit=130)
all_sand_weth_history, other_sand_weth_history = filter_swaps(sand_weth_history)
sand_weth_df = pool_history_to_df(all_sand_weth_history)
sand_weth_df

100%|██████████| 130/130 [03:24<00:00,  1.58s/it]


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,WETH,SAND,0.017650,126.469664,3.647599,2020-08-20 15:46:06,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x669f6505301592951630e2951b5132d29ca87e2c,0x37ed56f910e4cbb227e227808253ebb3284e165b2aab...
1,WETH,SAND,0.400000,2413.756323,178.145088,2020-08-20 15:56:12,0xbf06789ee859698ed737d6c9eb404791a85f5e9e,0xbf06789ee859698ed737d6c9eb404791a85f5e9e,0xa5c3510c8d2bdaefcef8d8a54b75c30492f7ef5ec440...
2,WETH,SAND,0.476309,2155.508531,212.249884,2020-08-20 15:57:40,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xb9a130e9c8f53d08e37a2d2183a4c87e0aa700ec,0xdb86739b7bfe561b0efb5168cf1bd0dd16ddb64982f1...
3,WETH,SAND,0.031877,123.596424,13.288490,2020-08-20 16:02:32,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7330707800b149effd2dcf37886181a01415664b,0x37f34948aa06656ba2b6771ff16122b73008ab4d1a90...
4,WETH,SAND,0.008500,32.612732,3.539984,2020-08-20 16:20:56,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x3e38f73779ea15d338cebe7fb68fa97fd22c212c,0xe11c12ff69938a56771c948e82f6f792d20cc78b3d14...
...,...,...,...,...,...,...,...,...,...
129989,WETH,SAND,0.153232,255.762458,712.025097,2021-11-13 23:18:28,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0xe66b31678d6c16e9ebf358268a790b763c133750,0x8311dd3f410e84189bd109a4f2b2ddd78d4fdd20bcd7...
129990,WETH,SAND,10.727217,17863.715000,49837.571981,2021-11-13 23:19:54,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x4da0eda15a164e78fddc0eddbee9295a78aa230e,0x42e5e167ffcfed5caf95ef2ddd1f5cf1215bad682997...
129991,WETH,SAND,8.590646,14247.190000,39906.498964,2021-11-13 23:22:30,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x4da0eda15a164e78fddc0eddbee9295a78aa230e,0x142c6574680ed4bf7179c23b3b2221b163c780e09cbe...
129992,WETH,SAND,0.212987,352.569454,989.397888,2021-11-13 23:22:30,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0xe66b31678d6c16e9ebf358268a790b763c133750,0x783a65f523139bc1113eeeee17effa018afada8c2ee3...


In [83]:
sand_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0x3dd49f67e9d5bc4c5e6634b3f70bfd9dc1b6bd74')
sand_weth_reserves_df = pool_reserves_to_df(sand_weth_reserves_history)
sand_weth_reserves_df

100%|██████████| 2/2 [00:02<00:00,  1.42s/it]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,4.620900e+04,6.473900,5.360654e+03,1.362405e+05,24.472452,2020-08-20
1,8.011954e+04,9.442597,7.270852e+03,1.862165e+05,26.147471,2020-08-21
2,8.079633e+04,10.412832,8.245477e+03,7.209562e+04,9.277340,2020-08-22
3,1.145013e+05,14.152003,1.110878e+04,5.906999e+04,7.473852,2020-08-23
4,1.199340e+05,14.999432,1.218586e+04,1.398868e+05,20.077351,2020-08-24
...,...,...,...,...,...,...
457,6.567291e+06,6288.435033,5.540371e+07,3.608024e+06,3480.449469,2021-11-20
458,6.817464e+06,6440.973519,5.536321e+07,2.214409e+06,2085.394378,2021-11-21
459,6.107998e+06,7401.983994,6.067011e+07,5.484632e+06,5632.281126,2021-11-22
460,6.113292e+06,7546.419056,6.561037e+07,8.077255e+06,10216.035409,2021-11-23


In [84]:
sand_weth_mints_history = get_pool_v2_mints(contract_id='0x3dd49f67e9d5bc4c5e6634b3f70bfd9dc1b6bd74', range_limit=2)
sand_weth_mints_df = pool_mints_to_df(sand_weth_mints_history)
sand_weth_mints_df

100%|██████████| 2/2 [00:02<00:00,  1.02s/it]


,amount0,amount1,amountUSD,liquidity,timestamp
0,1250.000000,0.168750,69.715655,14.523688,2020-08-20 15:26:19
1,3000.000000,0.405000,166.878223,34.856850,2020-08-20 15:29:38
2,14171.000000,2.032416,842.268420,169.702062,2020-08-20 15:49:30
3,1250.000000,0.318767,265.218003,19.952492,2020-08-20 16:02:23
4,10000.000000,2.470893,2058.665607,157.087106,2020-08-20 16:42:48
...,...,...,...,...,...
1995,6501.878843,0.498644,591.403567,53.076827,2020-12-01 17:38:53
1996,119.122908,0.009126,10.869397,0.971931,2020-12-01 19:45:46
1997,1419.066263,0.107953,129.199030,11.537341,2020-12-01 20:11:12
1998,2899.167241,0.220488,263.377507,23.567692,2020-12-01 20:15:13


In [85]:
sand_weth_burns_history = get_pool_v2_burns(contract_id='0x3dd49f67e9d5bc4c5e6634b3f70bfd9dc1b6bd74', range_limit=2)
sand_weth_burns_df = pool_burns_to_df(sand_weth_burns_history)
sand_weth_burns_df

100%|██████████| 2/2 [00:01<00:00,  1.03it/s]


,amount0,amount1,amountUSD,liquidity,timestamp
0,14443.767380,1.752435,1409.746715,157.087106,2020-08-24 17:33:44
1,18821.089713,2.694975,2151.411546,221.784224,2020-08-25 03:49:23
2,14121.242828,3.530601,2685.764707,219.172492,2020-08-27 15:22:02
3,17412.625149,3.007004,2404.186361,223.913860,2020-08-29 17:24:15
4,13464.195804,2.271845,1826.248541,171.121556,2020-08-30 06:57:30
...,...,...,...,...,...
1995,4897.017872,1.199387,8273.019079,60.000000,2021-09-09 23:04:57
1996,2075.032593,0.498905,3490.012333,25.184565,2021-09-10 02:36:35
1997,4589.506444,1.116136,7673.015477,56.018826,2021-09-10 06:33:41
1998,3634.904700,0.868581,5863.513224,43.976966,2021-09-10 08:57:51


### Writing swaps, mints, burns, reserves into pickle files

In [86]:
sand_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\sand_weth_swaps.pkl")
sand_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\sand_weth_reserves.pkl")
sand_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\sand_weth_mints.pkl")
sand_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\sand_weth_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [87]:
sand_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\sand_weth_swaps.pkl")
sand_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\sand_weth_reserves.pkl")
sand_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\sand_weth_mints.pkl")
sand_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\sand_weth_burns.pkl")

# ALICE/ETH pool - NFT

In [88]:
alice_weth_history = get_pool_v2_history(contract_id='0x30bc873193466dc59fc08159460c49f26f609795', range_limit=10)
all_alice_weth_history, other_alice_weth_history = filter_swaps(alice_weth_history)
alice_weth_df = pool_history_to_df(all_alice_weth_history)
alice_weth_df

100%|██████████| 10/10 [00:11<00:00,  1.16s/it]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,ALICE,WETH,12.839028,0.174107,284.992234,2021-03-16 08:37:32,0x0000000000c9b1dde987a6abe20a2af8d9e636d9,0x0000000000c9b1dde987a6abe20a2af8d9e636d9,0x2cdf6512b85e1c20f548ba8c318846a1dd8dd2917ebd...
1,ALICE,WETH,2.000000,0.021126,36.733660,2021-03-16 12:04:44,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x73191438977d7ff2d8acb9df6bc49cac321949ab4c6e...
2,ALICE,WETH,14.224945,0.120341,224.296642,2021-03-17 07:37:35,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0xf5ef67632cd2256d939702a126fe2c047d0a07bf,0x272eaf073e8ea7d7b2cc263db9b05ae91684866d64fa...
3,WETH,ALICE,0.187000,20.060961,329.448128,2021-03-17 11:36:13,0x11111112542d85b3ef69ae05771c2dccff4faa26,0xe8c423de4fa5e56537517944a77ff94cee6fb8e2,0x8301c0fd62a4f98a3306837835556300874eefc6e7ce...
4,ALICE,WETH,10.000000,0.105862,169.483118,2021-03-17 12:26:52,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x3eac79213f0d93f912851dfa46fafc82469eda6ac4b1...
...,...,...,...,...,...,...,...,...,...
8426,WETH,ALICE,1.750000,268.673810,7479.967587,2021-11-25 06:46:23,0x642bffb61457fefa2fdd57c13aa24a2eaf0e5eea,0xdd75cd55c1367b3d2d928c6181eea46999d24a72,0x520703fd9962956401290da1be9f02836d0f1d059960...
8427,WETH,ALICE,0.294813,44.502987,1261.938032,2021-11-25 07:17:09,0xe66b31678d6c16e9ebf358268a790b763c133750,0x0225ca88e84784211297e6bfaecdf737842dec2e,0xbe0ec899a9286f086e146d2eecb6a6122d105a059ee9...
8428,WETH,ALICE,0.024781,3.731008,106.386760,2021-11-25 07:25:05,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xa074507c9ec72aa1d0f716b6482d76e49330777f4e62...
8429,ALICE,WETH,35.953357,0.236957,1016.785322,2021-11-25 07:42:28,0x0000000000000eb4ec62758aae93400b3e5f7f18,0x55d5c232d921b9eaa6b37b5845e439acd04b4dba,0x68828c86b4d30c037e241338f958bf494a227b7b725a...


In [89]:
alice_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0x30bc873193466dc59fc08159460c49f26f609795')
alice_weth_reserves_df = pool_reserves_to_df(alice_weth_reserves_history)
alice_weth_reserves_df

100%|██████████| 2/2 [00:00<00:00,  2.93it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,6.000000,0.101598,3.673550e+02,0.000000,0.000000,2021-03-15
1,67.100212,0.689707,2.430779e+03,14.839028,0.195232,2021-03-16
2,68.634374,0.675505,2.232419e+03,46.915728,0.438203,2021-03-17
3,74.634374,0.621349,2.098103e+03,6.000000,0.054155,2021-03-22
4,85.550512,0.544140,1.817053e+03,106.134748,1.034504,2021-03-23
...,...,...,...,...,...,...
246,22848.051422,97.433199,8.331916e+05,41232.998887,197.500289,2021-11-21
247,21577.669556,103.220851,8.443882e+05,3717.518590,16.605229,2021-11-22
248,20056.083201,111.401124,9.685291e+05,21385.900979,116.369748,2021-11-23
249,18840.755150,118.996346,1.013472e+06,22159.528305,135.145361,2021-11-24


In [90]:
alice_weth_mints_history = get_pool_v2_mints(contract_id='0x30bc873193466dc59fc08159460c49f26f609795', range_limit=2)
alice_weth_mints_df = pool_mints_to_df(alice_weth_mints_history)
alice_weth_mints_df

100%|██████████| 2/2 [00:00<00:00,  3.94it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,6.000000,0.101598,367.355029,7.807616e-07,2021-03-15 10:52:49
1,46.261184,0.783341,2324.406045,6.019826e-06,2021-03-16 08:37:29
2,16737.068671,96.832368,339371.842500,1.267791e-03,2021-03-25 16:19:41
3,993.683353,7.264836,23374.408643,8.459013e-05,2021-03-25 19:34:30
4,2851.000000,20.025448,64780.308064,2.377853e-04,2021-03-26 02:52:05
5,7118.442317,50.000000,161393.357047,5.937079e-04,2021-03-26 02:54:47
6,4271.065390,30.000000,96854.620482,3.562247e-04,2021-03-26 02:56:50
7,100.000000,0.786183,2556.857781,8.822277e-06,2021-03-26 12:31:12
8,1075.901600,8.270369,27162.727663,9.384072e-05,2021-03-26 15:23:33
9,301.370000,2.437962,8134.923425,2.696432e-05,2021-03-26 20:41:38


In [91]:
alice_weth_burns_history = get_pool_v2_burns(contract_id='0x30bc873193466dc59fc08159460c49f26f609795', range_limit=2)
alice_weth_burns_df = pool_burns_to_df(alice_weth_burns_history)
alice_weth_burns_df

100%|██████████| 2/2 [00:00<00:00,  3.90it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,970.524283,7.451131,25321.238237,8.459013e-05,2021-03-27 04:44:20
1,44.370849,0.425158,1447.792493,4.315723e-06,2021-03-28 17:30:13
2,20.643398,0.192727,650.151355,1.981515e-06,2021-03-29 05:49:22
3,155.562830,1.505671,5140.720429,1.520242e-05,2021-03-29 08:42:17
4,57.815938,0.433081,1726.990931,4.965505e-06,2021-04-02 13:23:20
5,119.999999,0.832313,3487.019509,9.911754e-06,2021-04-03 15:06:29
6,6197.517242,37.706955,162681.978334,4.780017e-04,2021-04-10 11:04:23
7,16679.110145,98.610485,423748.268984,1.267791e-03,2021-04-11 02:51:04
8,1194.704911,7.557609,32169.567383,9.384072e-05,2021-04-12 15:23:30
9,192.252928,0.954394,4735.915000,1.336820e-05,2021-04-16 04:28:06


### Writing swaps, mints, burns, reserves into pickle files

In [92]:
alice_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\alice_weth_swaps.pkl")
alice_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\alice_weth_reserves.pkl")
alice_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\alice_weth_mints.pkl")
alice_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\alice_weth_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [93]:
alice_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\alice_weth_swaps.pkl")
alice_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\alice_weth_reserves.pkl")
alice_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\alice_weth_mints.pkl")
alice_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\alice_weth_burns.pkl")

# DOGE/ETH pool

In [94]:
doge_weth_history = get_pool_v2_history(contract_id='0xc0067d751fb1172dbab1fa003efe214ee8f419b6', range_limit=15)
all_doge_weth_history, other_doge_weth_history = filter_swaps(doge_weth_history)
doge_weth_df = pool_history_to_df(all_doge_weth_history)
doge_weth_df

100%|██████████| 15/15 [00:18<00:00,  1.61it/s]

list index out of range
list index out of range


100%|██████████| 15/15 [00:18<00:00,  1.22s/it]


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,WETH,DOGE,1.000000,131189.389055,1237.901160,2021-01-18 21:14:49,0xb89fa54d5d7bcb78e121f9fa85cb47242355c68e,0xb89fa54d5d7bcb78e121f9fa85cb47242355c68e,0xebb57fcd4f7c7c8c155d6e5a80f3212c7f2d80640bb3...
1,WETH,DOGE,0.007725,1000.000000,9.508499,2021-01-18 21:18:57,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x79d774e686d5d3cc1660ea4c88a11702b637852d,0x340d7456b432dd37e467d7a6fe3d6b3843e627d14d20...
2,WETH,DOGE,0.041338,5347.743915,50.895183,2021-01-18 21:29:35,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x72478dcaaa7fc87718505daf7cb530ae9612e53a,0x0ea8f1609f74a727484e53da819e12a27159df3fdf6d...
3,WETH,DOGE,0.025787,3333.000000,31.799822,2021-01-18 21:31:54,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x212919050e84572e3feb9f7d2d47d061c46fa4da,0xaddc432df998f8884ca9aa2bbe325f8df121cd5c3be6...
4,WETH,DOGE,0.300000,38736.888276,372.869812,2021-01-18 23:33:09,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x25770bc4f6b79ec5625c8e61bc92360caae23754,0xa834c8fbd15a81a708243b8f393877846e05b1e871b6...
...,...,...,...,...,...,...,...,...,...
12171,DOGE,WETH,38241.829903,2.005249,8560.755544,2021-11-24 08:18:12,0xb2bbfd73971279a457514325f209eb63a0341ef2,0xb2bbfd73971279a457514325f209eb63a0341ef2,0xdb47f7f45ec9f7f5a5650ead3a206b2c3d61c0de4bf6...
12172,DOGE,WETH,100000.000000,5.181213,22293.015378,2021-11-24 09:58:15,0x1111111254fb6c44bac0bed2854e76f90643097d,0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852,0xc3a2b979eae3d49d64d85863c9578c987f9732089aa1...
12173,DOGE,WETH,40517.171810,2.073944,8846.473981,2021-11-25 00:15:15,0xb2bbfd73971279a457514325f209eb63a0341ef2,0xb2bbfd73971279a457514325f209eb63a0341ef2,0x5bd8708fd3f27c18f3ed96f9ec7f8e1cbcb93cd7eb02...
12174,WETH,DOGE,0.328728,6402.430383,1414.967283,2021-11-25 02:40:41,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x056b7c88a79ac4e3f2c3f3d44f3e9af261f1ca97,0xbd989c3c99a99c90c63d1353b6a7bf2fc5b20e3d9425...


In [95]:
doge_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0xc0067d751fb1172dbab1fa003efe214ee8f419b6')
doge_weth_reserves_df = pool_reserves_to_df(doge_weth_reserves_history)
doge_weth_reserves_df

100%|██████████| 2/2 [00:00<00:00,  2.76it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,5.815876e+07,449.362514,1.117025e+06,1.796070e+05,1.374850,2021-01-18
1,1.053052e+08,690.380932,1.924844e+06,8.014429e+06,56.096709,2021-01-19
2,1.048894e+08,693.127828,1.756477e+06,4.933253e+05,3.253104,2021-01-22
3,1.068894e+08,680.197323,1.847699e+06,2.020000e+06,13.058171,2021-01-25
4,1.083894e+08,670.812012,1.807903e+06,1.500000e+06,9.385311,2021-01-26
...,...,...,...,...,...,...
303,1.142605e+07,605.853906,5.283400e+06,5.698648e+04,3.027678,2021-11-21
304,1.142830e+07,605.751889,5.195916e+06,1.104917e+05,5.885732,2021-11-22
305,1.145479e+07,604.456965,5.227746e+06,6.612421e+05,35.976650,2021-11-23
306,1.159304e+07,597.270503,5.139708e+06,1.382418e+05,7.186462,2021-11-24


In [96]:
doge_weth_mints_history = get_pool_v2_mints(contract_id='0xc0067d751fb1172dbab1fa003efe214ee8f419b6', range_limit=2)
doge_weth_mints_df = pool_mints_to_df(doge_weth_mints_history)
doge_weth_mints_df

100%|██████████| 2/2 [00:00<00:00,  3.99it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,1.000000e+07,75.000000,9.236720e+04,0.273861,2021-01-18 21:14:49
1,1.604497e+07,123.805938,3.059632e+05,0.445688,2021-01-18 22:41:33
2,2.050000e+07,158.181727,3.911640e+05,0.569437,2021-01-18 22:49:49
3,1.179340e+07,91.000000,2.254626e+05,0.327590,2021-01-18 23:07:24
4,2.868813e+06,20.000000,5.296842e+04,0.075734,2021-01-19 02:19:07
5,2.135449e+06,14.000000,3.986648e+04,0.054666,2021-01-19 17:18:20
6,3.706187e+07,242.977582,6.774434e+05,0.948755,2021-01-19 18:23:45
7,5.307253e+03,0.050000,1.300749e+02,0.000163,2021-01-28 11:24:21
8,1.121247e+05,1.998984,5.392716e+03,0.004723,2021-01-28 23:07:30
9,2.298049e+03,0.119647,3.125922e+02,0.000165,2021-01-29 06:34:28


In [97]:
doge_weth_burns_history = get_pool_v2_burns(contract_id='0xc0067d751fb1172dbab1fa003efe214ee8f419b6', range_limit=2)
doge_weth_burns_df = pool_burns_to_df(doge_weth_burns_history)
doge_weth_burns_df

100%|██████████| 2/2 [00:00<00:00,  4.15it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,6.894842e+04,2.655201,7.543399e+03,0.004251,2021-01-29 14:15:04
1,3.185133e+03,0.084567,2.276126e+02,0.000163,2021-01-30 05:01:23
2,2.528739e+03,0.112135,3.666145e+02,0.000165,2021-02-08 06:45:07
3,3.643306e+03,0.169095,5.840740e+02,0.000243,2021-02-09 02:51:38
4,1.480797e+04,0.414450,1.655572e+03,0.000765,2021-02-20 18:36:01
5,3.023294e+03,0.096621,3.262478e+02,0.000166,2021-03-27 08:16:29
6,1.378062e+07,1857.350311,8.585947e+06,1.532300,2021-04-20 20:56:53
7,1.273397e+02,0.017883,1.225493e+02,0.000014,2021-05-04 10:54:47
8,1.511696e+03,0.318917,2.086123e+03,0.000209,2021-05-05 06:14:59
9,1.912582e+03,0.364567,2.542336e+03,0.000251,2021-05-07 21:46:23


### Writing swaps, mints, burns, reserves into pickle files

In [98]:
doge_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\doge_weth_swaps.pkl")
doge_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\doge_weth_reserves.pkl")
doge_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\doge_weth_mints.pkl")
doge_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\doge_weth_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [99]:
doge_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\doge_weth_swaps.pkl")
doge_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\doge_weth_reserves.pkl")
doge_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\doge_weth_mints.pkl")
doge_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\doge_weth_burns.pkl")

# ELON/ETH pool

In [ ]:
elon_weth_history = get_pool_v2_history(contract_id='0x7b73644935b8e68019ac6356c40661e1bc315860', range_limit=325)
all_elon_weth_history, other_elon_weth_history = filter_swaps(elon_weth_history)
elon_weth_df = pool_history_to_df(all_elon_weth_history)
elon_weth_df

In [103]:
elon_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0x7b73644935b8e68019ac6356c40661e1bc315860')
elon_weth_reserves_df = pool_reserves_to_df(elon_weth_reserves_history)
elon_weth_reserves_df

100%|██████████| 2/2 [00:02<00:00,  1.11s/it]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,3.650502e+13,715.602627,3.385606e+06,2.729964e+15,41193.755814,2021-04-23
1,2.459976e+13,1110.292501,4.916181e+06,5.911238e+14,22876.541746,2021-04-24
2,2.935363e+13,933.446347,4.311324e+06,1.858092e+14,6450.296998,2021-04-25
3,2.492029e+13,1127.543829,5.708305e+06,2.802904e+14,12044.298435,2021-04-26
4,1.801824e+13,1632.782457,8.678267e+06,1.908413e+14,13764.348242,2021-04-27
...,...,...,...,...,...,...
212,1.218692e+13,4848.289166,4.155711e+07,1.833893e+12,736.858975,2021-11-21
213,1.233739e+13,4791.356316,3.930956e+07,2.440711e+12,962.676185,2021-11-22
214,1.260622e+13,4693.219739,4.076270e+07,3.590066e+12,1358.401103,2021-11-23
215,1.374227e+13,4316.740588,3.678673e+07,1.203970e+13,3912.687800,2021-11-24


In [104]:
elon_weth_mints_history = get_pool_v2_mints(contract_id='0x7b73644935b8e68019ac6356c40661e1bc315860', range_limit=2)
elon_weth_mints_df = pool_mints_to_df(elon_weth_mints_history)
elon_weth_mints_df

100%|██████████| 2/2 [00:01<00:00,  1.73it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,5.000000e+14,4.000000e+01,9.489166e+04,1.414214e+08,2021-04-23 01:09:59
1,3.147686e+10,1.200000e-01,5.368835e+02,6.074555e+04,2021-04-23 04:19:58
2,2.215014e+10,1.330598e-01,5.855027e+02,5.357215e+04,2021-04-23 05:09:10
3,5.912952e+10,1.000000e+00,4.408241e+03,2.386108e+05,2021-04-23 06:58:15
4,1.477064e+10,2.611877e-01,1.151920e+03,6.094134e+04,2021-04-23 07:01:49
...,...,...,...,...,...
450,1.049260e+09,3.983514e-01,3.753509e+03,1.209722e+04,2021-11-11 10:57:29
451,9.999935e-02,3.557404e-11,3.045510e-07,1.112424e-06,2021-11-16 12:18:49
452,2.505998e+09,1.030895e+00,8.722660e+03,2.993524e+04,2021-11-17 22:48:18
453,1.441521e+08,5.999058e-02,5.121963e+02,1.729151e+03,2021-11-19 17:59:05


In [105]:
elon_weth_burns_history = get_pool_v2_burns(contract_id='0x7b73644935b8e68019ac6356c40661e1bc315860', range_limit=2)
elon_weth_burns_df = pool_burns_to_df(elon_weth_burns_history)
elon_weth_burns_df

100%|██████████| 2/2 [00:00<00:00,  2.21it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,1.832062e+10,2.123652e-01,9.208381e+02,60941.336199,2021-04-23 07:33:39
1,4.605134e+09,2.129189e-01,9.709752e+02,30021.200665,2021-04-23 12:22:04
2,9.950712e+09,4.110642e-01,1.859035e+03,60745.553498,2021-04-23 14:24:50
3,2.904929e+10,1.225029e+00,5.664796e+03,178166.748472,2021-04-23 17:10:29
4,1.763913e+09,1.063146e-01,4.990540e+02,12850.654109,2021-04-23 18:47:15
...,...,...,...,...,...
332,5.255243e+08,1.914844e-01,1.787955e+03,5928.531397,2021-11-12 22:15:53
333,1.038644e+08,3.743157e-02,3.516297e+02,1164.084061,2021-11-15 07:25:19
334,1.000054e-01,3.557190e-11,3.043630e-07,0.000001,2021-11-16 12:25:02
335,2.557944e+09,1.014147e+00,8.880452e+03,29935.242549,2021-11-20 19:36:28


### Writing swaps, mints, burns, reserves into pickle files

In [106]:
elon_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\elon_weth_swaps.pkl")
elon_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\elon_weth_reserves.pkl")
elon_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\elon_weth_mints.pkl")
elon_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\elon_weth_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [107]:
elon_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\elon_weth_swaps.pkl")
elon_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\elon_weth_reserves.pkl")
elon_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\elon_weth_mints.pkl")
elon_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\elon_weth_burns.pkl")

# SHIB/ETH pool

In [110]:
shib_weth_history = get_pool_v2_history(contract_id='0x811beed0119b4afce20d2583eb608c6f7af1954f', range_limit=1300)
all_shib_weth_history, other_shib_weth_history = filter_swaps(shib_weth_history)
shib_weth_df = pool_history_to_df(all_shib_weth_history)
shib_weth_df

 19%|█▉        | 251/1300 [07:48<26:46,  1.53s/it]

{'message': 'Failed to get entities from store: canceling statement due to conflict with recovery, query = "/* qid: 9a35776c806bd1b0-15274de755b3e523 */\\nselect \'Pair\' as entity, to_jsonb(c.*) as data from (\\nselect c.*, p.id::text as g$parent_id\\n/* child_type_d */ from rows from (unnest($1), unnest($2::text[])) as p(id, child_id), \\"sgd25626\\".\\"pair\\" c where c.\\"block_range\\" @> $3 and c.id = any($4::text[]) and c.id = p.child_id limit 1001) c\\n  -- binds: [[\\"0x009e8cbfdc8ebbceda18e00299bb9203a501ff37aab0a3b2f786d53bc98ebf99-0\\", \\"0x00a5fcd7ead7f70ce921836faaa92b87864558160674508fad7a612311054566-0\\", \\"0x00add938e8d0be5d816d5abeec1217729b10a604a644b42db5c7fb78ad371963-0\\", \\"0x0171e908e7b8261f45c368650602c0682f397e88df47cf89772c7682cc92dc64-0\\", \\"0x01b4379efb17628b5d8a16227332990e11d40c8c0b4230f79083b522cf650afe-0\\", \\"0x01c37cf4c7948e36e5a6109cea0dc1e7d70a8ffaaba578677fafaedffe9af13f-1\\", \\"0x021564e6f4705b17baa2cb4d5a2d98196ec21cab7fce91700d62bbf153e8

 83%|████████▎ | 1085/1300 [32:13<05:37,  1.57s/it]

{'message': 'Failed to get entities from store: canceling statement due to conflict with recovery, query = "/* qid: 9a35776c806bd1b0-fe06d3a3fe35e4f */\\nselect \'Pair\' as entity, to_jsonb(c.*) as data from (\\nselect c.*, p.id::text as g$parent_id\\n/* child_type_d */ from rows from (unnest($1), unnest($2::text[])) as p(id, child_id), \\"sgd25626\\".\\"pair\\" c where c.\\"block_range\\" @> $3 and c.id = any($4::text[]) and c.id = p.child_id limit 1001) c\\n  -- binds: [[\\"0x0039ba1684f80c372a30b4a987d7159d12232df0038e12f04e9556b91622f636-0\\", \\"0x005caaa84fcd9d263a0eec0985af1e0aedb2f1adc3a1eb95249512d7415effa7-0\\", \\"0x0066107488fff036eb220b3a5bc08aee376bf0c37e336803b0f6c40cc1d5f2af-0\\", \\"0x006ff68f4610c3767dddb5c194a4e843aad46aa66994f2b950dc0caba0bee76f-0\\", \\"0x00b353e504818b2d7bde471e6feaf400c335fc6cc0d5667790f06d2d15168965-0\\", \\"0x014c40a31f1f5c8eba0e19a8cdbc2d7685e5381e426f01d741b3b11ee7562d6f-0\\", \\"0x0189de0332e371c37dbb34ee9e2aef0be0f0cf713bb641d6f06fc32fbf7ac

 89%|████████▉ | 1159/1300 [34:25<03:37,  1.54s/it]

...

100%|█████████▉| 1299/1300 [36:38<00:00,  4.35it/s]

list index out of range


100%|██████████| 1300/1300 [36:39<00:00,  1.69s/it]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,WETH,SHIB,4.000000e-01,1.898354e+13,139.381551,2020-07-31 20:42:11,0xbf06789ee859698ed737d6c9eb404791a85f5e9e,0xbf06789ee859698ed737d6c9eb404791a85f5e9e,0x48c0c040f75e020602b81d637e959dd4c968c4358085...
1,WETH,SHIB,4.000000e-01,1.757930e+13,139.276591,2020-07-31 20:42:36,0xa23636dd7b1517e12088fb8c5b364c735111f979,0xa23636dd7b1517e12088fb8c5b364c735111f979,0xfd90214848d79fc1a202ed281de47bef6062e48eb109...
2,WETH,SHIB,2.000000e+00,7.144941e+13,746.446085,2020-07-31 20:43:55,0x2c334d73c68bbc45dd55b13c5dea3a8f84ea053c,0x2c334d73c68bbc45dd55b13c5dea3a8f84ea053c,0x30b7f84b294d259251d903c2ad43568aea44153ecb2d...
3,WETH,SHIB,1.000000e-01,2.990975e+12,34.282991,2020-07-31 20:44:34,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xc9ccd0d6311f2fe7c1d8d4c28396c700f47ce452,0x249a14b2f48cd5c2ec59507074a7ab96172efebff26a...
4,WETH,SHIB,1.000000e+00,2.320679e+13,353.606470,2020-07-31 20:45:35,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x3fd1ddf28f604cfe5fefa3adc38e38be2f7208d6,0x3ca177de2358a3e332c30e43e923dd3dfaeba41b3925...
...,...,...,...,...,...,...,...,...,...
1219040,WETH,SHIB,2.242036e-02,2.511547e+06,98.802657,2021-11-25 12:26:36,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0xe66b31678d6c16e9ebf358268a790b763c133750,0x8ba92252b8795bd4ca34b0ffcbec05540cdce56ec5e0...
1219041,SHIB,WETH,2.902039e+07,2.574612e-01,1136.363144,2021-11-25 12:29:36,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x80d972d2a62ba71814f4e08bd27f95e5d81d02a9,0xe54257f8230906f641a85b84a7201a360a1a6443f0a9...
1219042,SHIB,WETH,7.210137e+08,6.362204e+00,28111.523234,2021-11-25 12:31:41,0xa57bd00134b2850b2a1c55860c9e9ea100fdd6cf,0x56178a0d5f301baf6cf3e1cd53d9863437345bf9,0xf48b4c558b1ed18f29b843180688c39531a9f41d4609...
1219043,WETH,SHIB,4.725585e+00,5.330552e+08,20873.309947,2021-11-25 12:32:18,0x0000000000e4cc48a5427ca0b9ed03b5bc501c97,0xff64cb7ba5717a10dabc4be3a41acd2c2f95ee22,0xaf18588ac446772709e8da2f2795a84456d3114fb08e...


In [111]:
shib_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0x811beed0119b4afce20d2583eb608c6f7af1954f')
shib_weth_reserves_df = pool_reserves_to_df(shib_weth_reserves_history)
shib_weth_reserves_df

100%|██████████| 2/2 [00:03<00:00,  1.66s/it]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,3.475561e+13,149.972185,1.031592e+05,1.391419e+15,1092.291376,2020-07-31
1,4.995162e+13,128.090506,9.822113e+04,1.450646e+15,4047.355659,2020-08-01
2,7.443586e+13,103.836139,7.637526e+04,3.419726e+14,540.039674,2020-08-02
3,1.394962e+14,57.242026,4.402689e+04,4.040179e+14,341.670235,2020-08-03
4,1.390480e+14,58.146782,4.547281e+04,1.100752e+14,48.841749,2020-08-04
...,...,...,...,...,...,...
445,1.265763e+11,1299.166290,1.111057e+07,9.615640e+10,983.438625,2021-11-21
446,1.270637e+11,1330.759660,1.091785e+07,1.553581e+11,1605.441088,2021-11-22
447,1.327043e+11,1274.068341,1.107684e+07,1.506101e+11,1535.237383,2021-11-23
448,1.383499e+11,1223.556037,1.042443e+07,1.775581e+11,1607.949211,2021-11-24


In [112]:
shib_weth_mints_history = get_pool_v2_mints(contract_id='0x811beed0119b4afce20d2583eb608c6f7af1954f', range_limit=2)
shib_weth_mints_df = pool_mints_to_df(shib_weth_mints_history)
shib_weth_mints_df

100%|██████████| 2/2 [00:02<00:00,  1.16s/it]


,amount0,amount1,amountUSD,liquidity,timestamp
0,4.950000e+14,1.000000e+01,3.421447e+03,7.035624e+07,2020-07-31 20:42:11
1,2.385578e+11,4.610574e-01,3.156094e+02,3.258480e+05,2020-07-31 23:28:57
2,5.078508e+11,2.534585e+00,1.738039e+03,1.107577e+06,2020-08-01 00:28:52
3,2.850964e+12,6.645802e+00,4.552039e+03,4.205740e+06,2020-08-01 01:38:39
4,1.905753e+11,3.435528e-01,2.355079e+02,2.460249e+05,2020-08-01 02:06:23
...,...,...,...,...,...
1995,4.454220e+07,1.400000e-01,7.668379e+02,7.079650e+02,2021-06-07 00:27:04
1996,1.000000e+08,3.148006e-01,1.726741e+03,1.590509e+03,2021-06-07 00:37:47
1997,1.059140e+07,3.303798e-02,1.821972e+02,1.676571e+02,2021-06-07 01:23:25
1998,2.187981e+08,6.794273e-01,3.791229e+03,3.454987e+03,2021-06-07 02:19:48


In [113]:
shib_weth_burns_history = get_pool_v2_burns(contract_id='0x811beed0119b4afce20d2583eb608c6f7af1954f', range_limit=2)
shib_weth_burns_df = pool_burns_to_df(shib_weth_burns_history)
shib_weth_burns_df

100%|██████████| 2/2 [00:02<00:00,  1.23s/it]


,amount0,amount1,amountUSD,liquidity,timestamp
0,2.680203e+11,0.427597,294.125991,3.258480e+05,2020-08-01 01:53:35
1,2.566214e+12,7.721197,5482.467576,4.205740e+06,2020-08-01 10:11:37
2,7.284586e+11,1.888833,1325.914127,1.107577e+06,2020-08-01 11:33:36
3,1.907353e+11,0.389869,279.675095,2.568436e+05,2020-08-01 16:35:39
4,7.526120e+10,0.157526,113.293753,1.025480e+05,2020-08-01 16:45:55
...,...,...,...,...,...
1995,1.037841e+08,0.382732,1552.068697,1.525532e+03,2021-07-13 07:44:22
1996,2.715608e+06,0.009903,39.779427,3.969099e+01,2021-07-13 11:51:48
1997,4.053543e+07,0.147503,583.155949,5.916668e+02,2021-07-13 14:45:25
1998,8.424326e+06,0.030611,121.975048,1.228624e+02,2021-07-13 17:34:31


### Writing swaps, mints, burns, reserves into pickle files

In [114]:
shib_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\shib_weth_swaps.pkl")
shib_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\shib_weth_reserves.pkl")
shib_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\shib_weth_mints.pkl")
shib_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\shib_weth_burns.pkl")

### Reading swaps, mints, burns, reserves from pickle files

In [115]:
shib_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\shib_weth_swaps.pkl")
shib_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\shib_weth_reserves.pkl")
shib_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\shib_weth_mints.pkl")
shib_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\shib_weth_burns.pkl")

# XAUt/WETH (STO)

In [214]:
xaut_weth_history = get_pool_v2_history(contract_id='0x57fc8adbf7b583b929c51df0da16008651c1de78', range_limit=2)
all_xaut_weth_history, other_xaut_weth_history = filter_swaps(xaut_weth_history)
xaut_weth_df = pool_history_to_df(all_xaut_weth_history)
xaut_weth_df

100%|██████████| 2/2 [00:00<00:00,  3.59it/s]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,XAUt,WETH,0.050915,0.244277,96.390518,2020-08-28 17:23:19,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x0efb068354c10c070ddd64a0e8eaf8f054df7e26,0xe81173951e5ed16d45c057837a1689d5d49043107874...
1,XAUt,WETH,0.040765,0.192847,76.555744,2020-08-29 13:04:09,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2b5fc30e00eecec02345e39665a03425595d6e37541d...
2,XAUt,WETH,0.038743,0.181074,69.943045,2020-10-12 18:26:17,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xe89f3900de9cff5a77f483ac980e36fe5808341a9b87...
3,XAUt,WETH,0.007973,0.037006,15.295573,2020-10-24 16:45:17,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2cd2d275cdb237e696c60419109fac5f331484e1,0x9f4db3d4bd5df6908d6bf11645dc5fee4d660d9fc6f6...
4,XAUt,WETH,0.056348,0.259366,100.422505,2020-11-01 00:29:07,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x142b42c33eaed7aa0df1ba629e4932be82eccfd2c152...
5,XAUt,WETH,0.013219,0.060300,23.270928,2020-11-01 05:48:22,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xedaedd22e653c504ff6806bf61664292848eb26e,0x4271554075ad9ded6d7bf343eb5863ff6fb2efee93e5...
6,XAUt,WETH,0.266727,1.174339,533.796306,2020-11-16 17:17:10,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xa5e79baee540f000ef6f23d067cd3ac22c7d9fe6,0x8776b6b0a5aa4b5f5ca2dbcecad090dd13a9381ec62f...
7,XAUt,WETH,0.044158,0.186922,95.788356,2020-11-21 10:40:52,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0x22f9dcf4647084d6c31b2765f6910cd85c178c18,0xd0d5dc5a34de1a9e203dbaf98f5933a0ac7beb145a2d...
8,XAUt,WETH,1.295523,4.700667,2641.260732,2020-11-23 22:05:53,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x72fd50eff7b67343f9e7c2fb64e19576ef1fdd8aa27a...
9,XAUt,WETH,0.017000,0.053045,31.415128,2020-12-02 00:55:21,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xed09fedb24c4ed9df203ef98bb67718145abdaddfd21...


In [215]:
xaut_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0x57fc8adbf7b583b929c51df0da16008651c1de78')
xaut_weth_reserves_df = pool_reserves_to_df(xaut_weth_reserves_history)
xaut_weth_reserves_df

100%|██████████| 2/2 [00:01<00:00,  1.32it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,6.460066,31.331320,23810.828693,0.000000,0.000000,2020-08-25
1,6.510981,31.087043,24592.993633,0.050915,0.244277,2020-08-28
2,6.551746,30.894196,24568.233147,0.040765,0.192847,2020-08-29
3,6.590489,30.713122,23761.321805,0.038743,0.181074,2020-10-12
4,7.671801,35.752273,26337.908336,0.000000,0.000000,2020-10-16
5,7.679774,35.715266,29495.046610,0.007973,0.037006,2020-10-24
6,7.749341,35.395600,27301.811195,0.069567,0.319667,2020-11-01
7,8.016068,34.221261,31590.206597,0.266727,1.174339,2020-11-16
8,8.060226,34.034339,34925.265087,0.044158,0.186922,2020-11-21
9,9.355749,29.333672,35367.518706,1.295523,4.700667,2020-11-23


In [216]:
xaut_weth_mints_history = get_pool_v2_mints(contract_id='0x57fc8adbf7b583b929c51df0da16008651c1de78', range_limit=2)
xaut_weth_mints_df = pool_mints_to_df(xaut_weth_mints_history)
xaut_weth_mints_df

100%|██████████| 2/2 [00:00<00:00,  4.25it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,2.000000,9.700000,3809.102773,4.404543e-06,2020-08-25 09:17:35
1,1.123192,5.447481,4310.404854,2.473574e-06,2020-08-25 11:03:09
2,3.000000,14.550000,11031.125400,6.606815e-06,2020-08-25 21:35:49
3,0.336874,1.633839,1241.666742,7.418880e-07,2020-08-25 21:50:02
4,1.081312,5.039151,3712.230849,2.334217e-06,2020-10-16 12:03:43


In [217]:
xaut_weth_burns_history = get_pool_v2_burns(contract_id='0x57fc8adbf7b583b929c51df0da16008651c1de78', range_limit=2)
xaut_weth_burns_df = pool_burns_to_df(xaut_weth_burns_history)
xaut_weth_burns_df

 50%|█████     | 1/2 [00:00<00:00,  3.45it/s]

list index out of range


100%|██████████| 2/2 [00:00<00:00,  3.92it/s]

list index out of range


AttributeError: 'DataFrame' object has no attribute 'amount0'

In [218]:
xaut_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\xaut_weth_swaps.pkl")
xaut_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\xaut_weth_reserves.pkl")
xaut_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\xaut_weth_mints.pkl")
# xaut_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\xaut_weth_burns.pkl")

In [122]:
xaut_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\xaut_weth_swaps.pkl")
xaut_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\xaut_weth_reserves.pkl")
xaut_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\xaut_weth_mints.pkl")
# xaut_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\xaut_weth_burns.pkl")

# UMA/FEI (STO)

In [123]:
uma_fei_history = get_pool_v2_history(contract_id='0x0f024314588466416c8fc66013ba4d3ab2e4efe5', range_limit=2)
all_uma_fei_history, other_uma_fei_history = filter_swaps(uma_fei_history)
uma_fei_df = pool_history_to_df(all_uma_fei_history)
uma_fei_df

100%|██████████| 2/2 [00:00<00:00,  3.83it/s]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,FEI,UMA,29763.930711,2028.431898,30431.699497,2021-11-23 23:19:21,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,0x9545c5bc50defba38e1cb3d15efdbeb5ab81912e06fa...
1,FEI,UMA,14991.490675,1016.008106,15284.783538,2021-11-23 23:20:54,0x220bda5c8994804ac96ebe4df184d25e5c2196d4,0x220bda5c8994804ac96ebe4df184d25e5c2196d4,0x4756c2afc0a426a358fc48596b1105458296d08bae40...
2,FEI,UMA,14923.378191,1007.647675,15228.232087,2021-11-24 00:58:28,0x8698d9d5ea99809c00426484a80be2add4e54581,0x157dfa656fdf0d18e1ba94075a53600d81cb3a97,0xec06f0c0627a48c286675dab3b52b12162e276fdc1c7...
3,UMA,FEI,997.650000,14687.223241,14841.633179,2021-11-24 02:47:58,0x220bda5c8994804ac96ebe4df184d25e5c2196d4,0x94b0a3d511b6ecdb17ebf877278ab030acb0a878,0xf5bb16d17347ac05b7cb88082d2ccba93a35193b4372...
4,UMA,FEI,1448.948091,21236.062724,21465.493801,2021-11-24 03:32:20,0x8698d9d5ea99809c00426484a80be2add4e54581,0xbb2e5c2ff298fd96e166f90c8abacaf714df14f8,0xfee2383f6e7cc0d6cb0f53229dc636cb0dc756112e31...
5,UMA,FEI,2946.875181,42845.977124,43498.665360,2021-11-24 03:33:29,0x220bda5c8994804ac96ebe4df184d25e5c2196d4,0x220bda5c8994804ac96ebe4df184d25e5c2196d4,0x52ee629a9565a7b0ea163f3903abfc5d59c45542e463...
6,UMA,FEI,1775.024141,25587.611383,25599.227151,2021-11-24 06:55:44,0x27239549dd40e1d60f5b80b0c4196923745b1fd2,0x94b0a3d511b6ecdb17ebf877278ab030acb0a878,0xb3e6b5c9f401d00c1eafe1f0f44407c88e3ba947b5a1...
7,UMA,FEI,1957.490510,28028.298659,28273.346926,2021-11-24 10:02:48,0x27239549dd40e1d60f5b80b0c4196923745b1fd2,0x27239549dd40e1d60f5b80b0c4196923745b1fd2,0x74bf221fb2d4b73efcec4dc575eddae5b6f287dd13e6...
8,UMA,FEI,2125.189821,30206.672027,30404.757931,2021-11-24 10:09:55,0x27239549dd40e1d60f5b80b0c4196923745b1fd2,0x27239549dd40e1d60f5b80b0c4196923745b1fd2,0xc554eb631f2770a867f8e1234e4537c44c3a1df948cd...
9,UMA,FEI,1774.996477,25053.190464,25228.564157,2021-11-24 11:12:16,0x27239549dd40e1d60f5b80b0c4196923745b1fd2,0x27239549dd40e1d60f5b80b0c4196923745b1fd2,0xc91588ba1f2073a128fa75f08edf3d77ef411e621aca...


In [131]:
uma_fei_reserves_history = get_pool_v2_reserves_history(contract_id='0x0f024314588466416c8fc66013ba4d3ab2e4efe5')
uma_fei_reserves_df = pool_reserves_to_df(uma_fei_reserves_history)
uma_fei_reserves_df

100%|██████████| 2/2 [00:00<00:00,  4.34it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,546428.170291,8.053508e+06,1.643170e+07,3044.440004,44755.421386,2021-11-23
1,567215.947716,7.759310e+06,1.560182e+07,22803.072775,324044.643474,2021-11-24
2,563085.775212,7.817247e+06,1.577766e+07,24365.780519,343141.992167,2021-11-25


In [125]:
uma_fei_mints_history = get_pool_v2_mints(contract_id='0x0f024314588466416c8fc66013ba4d3ab2e4efe5', range_limit=2)
uma_fei_mints_df = pool_mints_to_df(uma_fei_mints_history)
uma_fei_mints_df

100%|██████████| 2/2 [00:00<00:00,  4.25it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,137536.474736,2.004641e+06,4.112547e+06,525082.193571,2021-11-23 23:17:47
1,137218.033475,2.000000e+06,4.103025e+06,523866.459081,2021-11-23 23:17:47
2,137500.000000,2.004110e+06,4.111457e+06,524942.941532,2021-11-23 23:17:47
3,137218.102084,2.000001e+06,4.103028e+06,523866.721014,2021-11-23 23:17:47


In [126]:
uma_fei_burns_history = get_pool_v2_burns(contract_id='0x0f024314588466416c8fc66013ba4d3ab2e4efe5', range_limit=2)
uma_fei_burns_df = pool_burns_to_df(uma_fei_burns_history)
uma_fei_burns_df

 50%|█████     | 1/2 [00:00<00:00,  3.68it/s]

list index out of range


100%|██████████| 2/2 [00:00<00:00,  2.69it/s]

list index out of range


AttributeError: 'DataFrame' object has no attribute 'amount0'

In [127]:
uma_fei_df.to_pickle(os.getcwd() + "\\pkl_stories\\uma_fei_swaps.pkl")
uma_fei_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\uma_fei_reserves.pkl")
uma_fei_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\uma_fei_mints.pkl")
# uma_fei_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\uma_fei_burns.pkl")

In [128]:
uma_fei_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\uma_fei_swaps.pkl")
uma_fei_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\uma_fei_reserves.pkl")
uma_fei_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\uma_fei_mints.pkl")
# uma_fei_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\uma_fei_burns.pkl")

# PERL/WETH (STO)

In [142]:
perl_weth_history = get_pool_v2_history(contract_id='0x9b8d2834ba1b3c31688e084ce9792baa0ca2e2f7', range_limit=2)
all_perl_weth_history, other_perl_weth_history = filter_swaps(perl_weth_history)
perl_weth_df = pool_history_to_df(all_perl_weth_history)
perl_weth_df

100%|██████████| 2/2 [00:02<00:00,  1.09s/it]


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,WETH,PERL,1.000000,408.979034,202.663354,2020-08-20 07:31:20,0x21e479e62603a3ea0b6dc687cb86b9938d39a3dd,0x21e479e62603a3ea0b6dc687cb86b9938d39a3dd,0x259920fbd70f1fb6081890bc61f3d04bb65f7fdb71be...
1,WETH,PERL,0.330000,9.444516,66.865797,2020-08-20 07:31:33,0x90b98c3db94f6d2a088a3d7e4c86e15a39f8ee0d,0x90b98c3db94f6d2a088a3d7e4c86e15a39f8ee0d,0xa2579b1d7d65a6196165219ce988979ed0c07a0ed4cb...
2,WETH,PERL,0.100000,2.058032,20.248495,2020-08-20 07:34:18,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x21f1d0daa58f49a0a03855966e79026d2001d00e,0x394e02caf2236dabe102ee16f563e438cc6dc4fa8c44...
3,WETH,PERL,0.111533,2.000000,22.576193,2020-08-20 07:37:55,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xc26da32095985cd03201103bf400fc8058621240,0xa7b8b130b5441fbf6aa77fe37c893ec9d9c76ae375e6...
4,PERL,WETH,2.058032,0.113905,23.020060,2020-08-20 07:41:29,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xb69d1b70c6d0059ff0ec29acf0dc39756dd5c38c90f2...
...,...,...,...,...,...,...,...,...,...
1529,PERL,WETH,70201.678416,1.730590,7353.996463,2021-11-24 23:14:13,0x220bda5c8994804ac96ebe4df184d25e5c2196d4,0x220bda5c8994804ac96ebe4df184d25e5c2196d4,0xbbe7cdeeaf0d55c054097ad4c8aa777a38714d402650...
1530,PERL,WETH,41395.008270,0.984122,4220.791473,2021-11-25 05:27:14,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xbf5ae133b9a0fc1a07952a7df2afa21f7f69ef58,0x6c1756fe2e08651e279d5a64b258d182135465cddc98...
1531,WETH,PERL,0.516205,21720.443798,2212.233302,2021-11-25 05:34:22,0xa1006d0051a35b0000f961a8000000009ea8d2db,0xa1006d0051a35b0000f961a8000000009ea8d2db,0x2a7fff98154aec80dec5c1a1cdc098322061e7eaef96...
1532,PERL,WETH,39265.021300,0.922462,3963.355895,2021-11-25 08:32:36,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xbf5ae133b9a0fc1a07952a7df2afa21f7f69ef58,0xc11b8124be745c228541257826e88e0ad7a46f815a53...


In [132]:
perl_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0x9b8d2834ba1b3c31688e084ce9792baa0ca2e2f7')
perl_weth_reserves_df = pool_reserves_to_df(perl_weth_reserves_history)
perl_weth_reserves_df

100%|██████████| 2/2 [00:00<00:00,  2.88it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,0.153498,2.958885e+02,62.347684,3.435517,1.087056e+03,2020-08-20
1,0.081605,5.573478e+02,33.663391,0.071893,2.614593e+02,2020-08-21
2,0.091605,4.966680e+02,33.233211,0.010000,6.067977e+01,2020-09-13
3,0.054551,8.350475e+02,19.107331,0.037054,3.383795e+02,2020-09-26
4,0.211676,4.124108e+03,75.807403,0.027090,4.691610e+02,2020-09-29
...,...,...,...,...,...,...
243,80.799774,2.850057e+06,697804.883043,212.875288,7.081302e+06,2021-11-21
244,76.170139,3.025013e+06,620158.020751,14.806463,5.753920e+05,2021-11-22
245,72.524184,3.178264e+06,631014.898615,9.120572,3.901169e+05,2021-11-23
246,74.652832,3.089421e+06,634461.717019,14.935974,5.796793e+05,2021-11-24


In [133]:
perl_weth_mints_history = get_pool_v2_mints(contract_id='0x9b8d2834ba1b3c31688e084ce9792baa0ca2e2f7', range_limit=2)
perl_weth_mints_df = pool_mints_to_df(perl_weth_mints_history)
perl_weth_mints_df

100%|██████████| 2/2 [00:00<00:00,  4.33it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,0.100000,4.500000e+02,40.532671,6.708204,2020-08-20 07:31:20
1,0.184215,2.819900e+03,64.783335,22.653158,2020-09-29 17:26:24
2,0.002946,6.350000e+01,1.300028,0.424167,2020-11-06 17:48:44
3,0.458411,1.366484e+04,209.795819,77.502604,2020-11-15 03:28:53
4,4.000000,5.134500e+04,7104.893039,433.261162,2021-03-17 16:13:30
5,0.709776,7.977305e+03,1720.852504,71.214938,2021-04-22 04:32:40
6,1.000000,1.917393e+04,3056.506451,130.067237,2021-05-03 04:26:02
7,2.427751,1.000000e+05,6326.208035,453.925072,2021-08-05 10:25:44
8,67.660000,2.412971e+06,352221.287482,11767.232788,2021-08-05 13:24:54
9,4.497954,1.654781e+05,24809.899564,794.511145,2021-08-05 14:17:20


In [134]:
perl_weth_burns_history = get_pool_v2_burns(contract_id='0x9b8d2834ba1b3c31688e084ce9792baa0ca2e2f7', range_limit=2)
perl_weth_burns_df = pool_burns_to_df(perl_weth_burns_history)
perl_weth_burns_df

100%|██████████| 2/2 [00:00<00:00,  4.44it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,0.168286,3152.146958,61.360317,22.653158,2020-10-16 20:45:28
1,0.512509,12226.335932,234.545693,77.502604,2020-11-15 03:31:26
2,4.955154,42139.409144,18322.008027,433.261162,2021-03-31 18:12:31
3,1.188370,40797.542986,7648.119458,201.282176,2021-08-25 23:15:04


In [135]:
perl_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\perl_weth_swaps.pkl")
perl_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\perl_weth_reserves.pkl")
perl_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\perl_weth_mints.pkl")
perl_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\perl_weth_burns.pkl")

In [141]:
perl_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\perl_weth_swaps.pkl")
perl_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\perl_weth_reserves.pkl")
perl_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\perl_weth_mints.pkl")
perl_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\perl_weth_burns.pkl")

# BPT/WETH (STO)

In [151]:
bpt_weth_history = get_pool_v2_history(contract_id='0x13e638b4f89740a1c2ff45d71f71ee28101cc1dc', range_limit=5)
all_bpt_weth_history, other_bpt_weth_history = filter_swaps(bpt_weth_history)
bpt_weth_df = pool_history_to_df(all_bpt_weth_history)
bpt_weth_df

100%|██████████| 5/5 [00:04<00:00,  1.16it/s]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,BPT,WETH,0.028860,5.899406,981.913051,2020-05-31 20:25:35,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0x96c3ba83d4aacebef71866ecaa8d57a5d681bf470e20...
1,WETH,BPT,0.100000,0.001160,23.833425,2020-05-31 20:34:44,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0x19bc2718782f2ae9d22c66d4d364394ecad1a771,0x8f4a0a3e964ff565c46d788ebef3ec25cc589a37b379...
2,WETH,BPT,0.050000,0.000559,11.843336,2020-05-31 20:38:21,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0x19bc2718782f2ae9d22c66d4d364394ecad1a771,0xe31f8cb2aa234d1f690c063577702a9be98c10670f51...
3,WETH,BPT,1.000000,0.008956,263.136013,2020-05-31 21:03:51,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0x2593ca08fda3c2526409493cd1ffa3f656d0b7c6,0x712c35f27ffe151a02846fd3748ff034e6d123b45ca0...
4,BPT,WETH,0.010674,0.213296,25.501870,2020-06-06 07:04:33,0x693c188e40f760ecf00d2946ef45260b84fbc43e,0x693c188e40f760ecf00d2946ef45260b84fbc43e,0xa4526e81d57c9a1f5dbeede9b333ac9f7806e88260f0...
...,...,...,...,...,...,...,...,...,...
3151,WETH,BPT,0.004000,89828.981912,6.613954,2021-08-24 00:59:34,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xce6e51ae6a876feb78df6bd7ce224935d249d61f,0x434eae879295625399358b088daffab5e28fe426ce3b...
3152,WETH,BPT,0.005000,82946.747246,8.229742,2021-08-24 01:27:16,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xce6e51ae6a876feb78df6bd7ce224935d249d61f,0x6810a641f416cb9ad102ac82a73ad3c375e02f75e137...
3153,BPT,WETH,308326.263648,0.013275,22.034868,2021-08-24 10:46:30,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,0xfeb0ca3b8abb121c3abeecd2729b46a38ed4d972af87...
3154,WETH,BPT,0.002000,68782.696426,3.064949,2021-09-20 19:58:11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x03711e1f9e1c1767690958a1fc6f4a88ec74253c,0x3228d95f8ced3dd842d050c83a44d980afb713a665c9...


In [144]:
bpt_weth_reserves_history = get_pool_v2_reserves_history(contract_id='0x13e638b4f89740a1c2ff45d71f71ee28101cc1dc')
bpt_weth_reserves_df = pool_reserves_to_df(bpt_weth_reserves_history)
bpt_weth_reserves_df

100%|██████████| 2/2 [00:01<00:00,  1.91it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,8.600000e-17,1.174100e-14,5.524124e-12,0.039536,7.049406,2020-05-31
1,3.568712e-01,1.034103e+00,4.988966e+02,0.704712,4.782504,2020-06-06
2,2.933035e-01,1.278057e+00,3.108033e+02,1.694110,7.916374,2020-06-07
3,9.057933e-01,5.475164e+00,2.675147e+03,1.948616,11.165002,2020-06-08
4,7.743387e+00,3.873774e+01,1.891492e+04,7.808211,44.955769,2020-06-09
...,...,...,...,...,...,...
142,7.242572e+05,2.311737e-02,4.604234e+01,394952.489876,0.012186,2021-06-23
143,7.503883e+05,2.240183e-02,7.196929e+01,881199.884171,0.067534,2021-08-20
144,7.975512e+05,2.112689e-02,7.013638e+01,569489.586768,0.025275,2021-08-24
145,7.287685e+05,2.312689e-02,7.088274e+01,68782.696426,0.002000,2021-09-20


In [145]:
bpt_weth_mints_history = get_pool_v2_mints(contract_id='0x13e638b4f89740a1c2ff45d71f71ee28101cc1dc', range_limit=2)
bpt_weth_mints_df = pool_mints_to_df(bpt_weth_mints_history)
bpt_weth_mints_df

100%|██████████| 2/2 [00:00<00:00,  4.11it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,0.020000,10.000000,2354.500343,0.447214,2020-05-31 20:08:08
1,0.001831,0.250000,59.780438,0.021293,2020-06-06 07:04:20
2,0.340693,1.000000,239.076967,0.580127,2020-06-06 07:15:57
3,0.174360,1.000000,486.384907,0.409298,2020-06-08 09:55:18
4,0.167803,1.000000,486.499650,0.401232,2020-06-08 10:06:02
5,0.167803,1.000000,486.602570,0.401232,2020-06-08 10:17:14
6,0.137937,1.000000,486.689204,0.363726,2020-06-08 10:21:09
7,0.571854,2.500000,1215.168223,1.168421,2020-06-09 08:06:05
8,2.287418,10.000000,4860.672890,4.673685,2020-06-09 08:08:58
9,4.346094,19.000000,9201.905567,8.880001,2020-06-09 08:35:19


In [146]:
bpt_weth_burns_history = get_pool_v2_burns(contract_id='0x13e638b4f89740a1c2ff45d71f71ee28101cc1dc', range_limit=2)
bpt_weth_burns_df = pool_burns_to_df(bpt_weth_burns_history)
bpt_weth_burns_df

100%|██████████| 2/2 [00:00<00:00,  3.90it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,0.038184,5.250594,2461.568018,0.447214,2020-05-31 21:06:15
1,0.417425,2.000000,945.674491,0.887479,2020-06-13 10:07:00
2,0.119339,0.582595,275.554580,0.256009,2020-06-13 16:50:17
3,3.280877,16.797741,7885.957027,7.204152,2020-06-14 10:14:07
4,2.659988,11.818349,5190.763404,5.436221,2020-06-15 06:32:10
5,2.575563,13.775960,6217.870796,5.772561,2020-06-15 17:54:35
6,0.007609,0.038823,17.953201,0.016500,2020-06-22 02:10:53
7,2.714016,13.339686,6494.472194,5.771805,2020-06-22 18:27:18
8,0.228369,1.126151,543.369184,0.485606,2020-06-23 16:38:16
9,4.187742,20.707012,9628.537045,8.880001,2020-06-25 16:48:59


In [147]:
bpt_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\bpt_weth_swaps.pkl")
bpt_weth_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\bpt_weth_reserves.pkl")
bpt_weth_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\bpt_weth_mints.pkl")
bpt_weth_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\bpt_weth_burns.pkl")

In [148]:
bpt_weth_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\bpt_weth_swaps.pkl")
bpt_weth_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\bpt_weth_reserves.pkl")
bpt_weth_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\bpt_weth_mints.pkl")
bpt_weth_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\bpt_weth_burns.pkl")

# uSTONKS_APR21/USDC (STO)

In [152]:
stonks_usdc_history = get_pool_v2_history(contract_id='0xedf187890af846bd59f560827ebd2091c49b75df', range_limit=2)
all_stonks_usdc_history, other_stonks_usdc_history = filter_swaps(stonks_usdc_history)
stonks_usdc_df = pool_history_to_df(all_stonks_usdc_history)
stonks_usdc_df

100%|██████████| 2/2 [00:00<00:00,  2.53it/s]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,USDC,uSTONKS_APR21,2159.220526,10.000000,2157.435115,2021-03-09 15:01:21,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x9670d11c6bdde4d239f699ac71dd77075c35603d,0xeaf57e2488fdd6b12b0d01ba29f687f2523efe2f1126...
1,USDC,uSTONKS_APR21,909.722591,4.178164,910.537242,2021-03-09 15:03:07,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x64e78ba4baca646f77a6bf7bcbcc5a31fc066ef2,0xdaa555d301b25d8dec1d814a1edf263b25ecf703c00e...
2,USDC,uSTONKS_APR21,8881.277097,40.000000,8903.658224,2021-03-09 15:41:02,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xd2a78bb82389d30075144d17e782964918999f7f,0x749f55e09dfd5ec051ad880919f44c9816b708718d18...
3,USDC,uSTONKS_APR21,225.978716,1.000000,225.626999,2021-03-09 16:10:17,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xf3239400cd2135cf5f99e191018eeb2a6d01eb76,0xbec0e4637d4a85ecbae54044c8b49a26929eca1a91d4...
4,USDC,uSTONKS_APR21,452.519620,2.000000,452.148230,2021-03-09 16:52:44,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x9ebc8ad4011c7f559743eb25705ccf5a9b58d0bc,0xadf350a8ae7c9576db9f875e88081856085159359245...
...,...,...,...,...,...,...,...,...,...
170,uSTONKS_APR21,USDC,0.085158,19.914283,19.936291,2021-06-06 21:24:34,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,0x1f6998d775b94a8dd2cbc1440aa8b98bc5436037fe8d...
171,uSTONKS_APR21,USDC,2.000000,456.462039,451.857246,2021-06-16 21:32:05,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,0xaec738f4941021329b4f22a5e3f4ac8b350094da6fea...
172,uSTONKS_APR21,USDC,14.194925,2628.472904,2408.735419,2021-06-16 22:20:28,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0x0990fd97223d006eae1f655e82467fa0ec5f0890,0x5ef7d1d1afae0c0ab474dd86361264bbc984686a4311...
173,USDC,uSTONKS_APR21,1757.054740,10.000000,1873.066834,2021-06-17 14:21:41,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x4481461f5d2cea356855917cd7c213001bbfa087,0xb0dd839a329ed3e0ff8339daacf3a41935ab35cd34b9...


In [153]:
stonks_usdc_reserves_history = get_pool_v2_reserves_history(contract_id='0xedf187890af846bd59f560827ebd2091c49b75df')
stonks_usdc_reserves_df = pool_reserves_to_df(stonks_usdc_reserves_history)
stonks_usdc_reserves_df

100%|██████████| 2/2 [00:00<00:00,  4.06it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,2140.000000,10.000000,2.140008e+03,0.000000,0.000000,2021-03-03
1,201160.000000,940.000000,2.010901e+05,0.000000,0.000000,2021-03-04
2,326686.649424,1526.573126,6.540885e+05,0.000000,0.000000,2021-03-08
3,772987.580097,3113.030262,1.548919e+06,44927.128659,192.385929,2021-03-09
4,964121.731448,3612.702829,1.927069e+06,68772.248216,266.364921,2021-03-10
...,...,...,...,...,...,...
63,77250.898244,329.266456,1.544002e+05,0.000000,0.000000,2021-06-05
64,77230.983961,329.351614,1.544204e+05,19.914283,0.085158,2021-06-06
65,19195.283406,81.858307,3.838067e+04,0.000000,0.000000,2021-06-09
66,12938.376910,83.858307,2.587076e+04,3084.934943,16.194925,2021-06-16


In [154]:
stonks_usdc_mints_history = get_pool_v2_mints(contract_id='0xedf187890af846bd59f560827ebd2091c49b75df', range_limit=2)
stonks_usdc_mints_df = pool_mints_to_df(stonks_usdc_mints_history)
stonks_usdc_mints_df

100%|██████████| 2/2 [00:00<00:00,  2.83it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,2140.000000,10.000000,2140.007877,1.462864e-10,2021-03-03 02:28:49
1,199020.000000,930.000000,198950.834595,1.360473e-08,2021-03-04 21:22:02
2,6756.649335,31.573127,13513.202827,4.618750e-10,2021-03-08 15:03:10
3,118770.000089,554.999999,237800.008546,8.118950e-09,2021-03-08 15:28:01
4,8667.000012,40.499999,17329.377349,5.924639e-10,2021-03-09 12:29:59
...,...,...,...,...,...
164,340.201166,1.299999,679.540116,2.102001e-11,2021-03-25 18:51:51
165,18742.306377,72.616696,37533.745868,1.166055e-09,2021-03-26 05:41:50
166,1262.832587,4.750000,2524.775878,7.740665e-11,2021-04-04 04:07:54
167,49.851846,0.187628,99.725424,3.056655e-12,2021-04-07 05:04:04


In [155]:
stonks_usdc_burns_history = get_pool_v2_burns(contract_id='0xedf187890af846bd59f560827ebd2091c49b75df', range_limit=2)
stonks_usdc_burns_df = pool_burns_to_df(stonks_usdc_burns_history)
stonks_usdc_burns_df

100%|██████████| 2/2 [00:00<00:00,  2.69it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,2176.015236,9.999998,4356.427361,1.475114e-10,2021-03-09 15:26:40
1,2271.214859,9.915943,4538.273084,1.500632e-10,2021-03-09 17:02:02
2,20026.726443,80.539183,39951.044276,1.269865e-09,2021-03-09 23:17:27
3,1093.486745,4.397549,2181.339222,6.933637e-11,2021-03-09 23:18:21
4,133.183882,0.535021,266.250195,8.440200e-12,2021-03-10 07:29:36
...,...,...,...,...,...
129,240.569900,1.158706,481.032553,1.668136e-11,2021-05-24 07:27:29
130,3164.520379,15.241929,6328.657926,2.194311e-10,2021-06-02 11:44:27
131,300.523456,1.280920,600.651467,1.959890e-11,2021-06-05 07:25:09
132,58035.700555,247.493307,116041.478759,3.785825e-09,2021-06-09 01:25:20


In [156]:
stonks_usdc_df.to_pickle(os.getcwd() + "\\pkl_stories\\stonks_usdc_swaps.pkl")
stonks_usdc_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\stonks_usdc_reserves.pkl")
stonks_usdc_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\stonks_usdc_mints.pkl")
stonks_usdc_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\stonks_usdc_burns.pkl")

In [157]:
stonks_usdc_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\stonks_usdc_swaps.pkl")
stonks_usdc_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\stonks_usdc_reserves.pkl")
stonks_usdc_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\stonks_usdc_mints.pkl")
stonks_usdc_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\stonks_usdc_burns.pkl")

# mAMZN/UST (STO)

In [159]:
amzn_ust_history = get_pool_v2_history(contract_id='0x0ae8cb1f57e3b1b7f4f5048743710084aa69e796', range_limit=3)
all_amzn_ust_history, other_amzn_ust_history = filter_swaps(amzn_ust_history)
amzn_ust_df = pool_history_to_df(all_amzn_ust_history)
amzn_ust_df

100%|██████████| 3/3 [00:02<00:00,  1.22it/s]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,UST,mAMZN,2350.000000,0.726130,1289.197989,2020-12-03 19:08:10,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xf53cab0c6f25f48f985d1e2c40e03fc7c1963364,0xf5c5766c2684fdd0997bf77520eb59828466e17f263b...
1,UST,mAMZN,4300.000000,1.300860,2326.242409,2020-12-03 19:09:07,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x4add3b2277e1a79e4cf6b52ccb26b2c80b37f142,0xb10cd689a829a7e356b73596d6fca504a11421d9387f...
2,UST,mAMZN,2884.657644,0.853276,1532.971308,2020-12-03 19:16:53,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xdec08cb92a506b88411da9ba290f3694be223c26,0x746b2fc227338792f29f6f22f4c5e06f21a12183e973...
3,mAMZN,UST,0.400000,1350.538418,719.845223,2020-12-03 19:21:42,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xdec08cb92a506b88411da9ba290f3694be223c26,0x215b96593f26c2b770b98fa8c89ad72585395c5cd994...
4,mAMZN,UST,0.400000,1339.406326,714.389322,2020-12-03 19:22:29,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xdec08cb92a506b88411da9ba290f3694be223c26,0x7d96885b515444001a8d60e19fc600aab4cafcf1ab54...
...,...,...,...,...,...,...,...,...,...
1621,UST,mAMZN,0.175045,0.000047,0.087596,2021-11-22 11:21:44,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x8ae48a768f8270e8349f2d4f1511311a1143e9ed,0xd6e9be6171f0fd526b15d373767e4695302d36037443...
1622,mAMZN,UST,10.989001,40681.790703,20471.767499,2021-11-22 15:08:58,0x1111111254fb6c44bac0bed2854e76f90643097d,0x5690df030163680f4da2046798614c3f5c24f64a,0x48c20eca5b2fba7e0fe38f96bffe9d76f3fd3dd7c070...
1623,UST,mAMZN,25000.000000,6.727834,12520.239419,2021-11-24 15:41:08,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x657e21723ae99c05c9c2a6032343c7865bcb1940,0xf5f03b8ddad1286f17c534edc42f9db3d29499dc2aa6...
1624,mAMZN,UST,8.857576,32681.237150,16384.640643,2021-11-24 15:43:08,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x9fb7e5f71517c9b0059e3200b51bbd1477db333b,0xe37ab2f795bee67c14075716c8d1a122c25bb67691b2...


In [160]:
amzn_ust_reserves_history = get_pool_v2_reserves_history(contract_id='0x0ae8cb1f57e3b1b7f4f5048743710084aa69e796')
amzn_ust_reserves_df = pool_reserves_to_df(amzn_ust_reserves_history)
amzn_ust_reserves_df

100%|██████████| 2/2 [00:01<00:00,  1.84it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,158.952192,5.641813e+05,5.653179e+05,16.852962,59216.161844,2020-12-03
1,177.710358,5.563377e+05,5.640840e+05,46.880551,163757.007667,2020-12-04
2,171.565135,5.634286e+05,5.822974e+05,4.142479,13322.279504,2020-12-05
3,169.159847,5.858983e+05,5.858964e+05,16.177000,55693.169813,2020-12-06
4,231.699170,7.406623e+05,7.386575e+05,11.446256,38683.434061,2020-12-07
...,...,...,...,...,...,...
311,1892.112264,7.076524e+06,7.128850e+06,1.177746,4394.303980,2021-11-20
312,1885.471313,7.101603e+06,7.106725e+06,6.651409,25039.229344,2021-11-21
313,1911.160752,7.055549e+06,7.091617e+06,19.079714,70935.229694,2021-11-22
314,1917.697714,7.079682e+06,7.088240e+06,0.000000,0.000000,2021-11-23


In [161]:
amzn_ust_mints_history = get_pool_v2_mints(contract_id='0x0ae8cb1f57e3b1b7f4f5048743710084aa69e796', range_limit=2)
amzn_ust_mints_df = pool_mints_to_df(amzn_ust_mints_history)
amzn_ust_mints_df

100%|██████████| 2/2 [00:01<00:00,  1.60it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,96.964393,310524.777000,0.000000,5487.243982,2020-12-03 16:38:49
1,0.726130,2425.916406,2624.945114,41.969274,2020-12-03 19:10:17
2,1.300860,4346.019834,4574.328112,75.187792,2020-12-03 19:14:47
3,0.282262,949.919644,1104.382118,16.373538,2020-12-03 20:16:18
4,0.588902,2006.229668,2404.438388,34.370103,2020-12-03 20:19:26
...,...,...,...,...,...
370,0.000047,0.174520,0.174667,0.002830,2021-11-22 11:21:44
371,6.641899,24520.304608,24645.652915,399.931440,2021-11-22 18:01:38
372,6.536962,24132.901841,24162.073827,393.612817,2021-11-23 05:41:18
373,6.727834,24783.084553,24824.455499,404.656477,2021-11-24 15:59:19


In [162]:
amzn_ust_burns_history = get_pool_v2_burns(contract_id='0x0ae8cb1f57e3b1b7f4f5048743710084aa69e796', range_limit=2)
amzn_ust_burns_df = pool_burns_to_df(amzn_ust_burns_history)
amzn_ust_burns_df

100%|██████████| 2/2 [00:00<00:00,  2.48it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,0.696842,2.528311e+03,2.756019e+03,41.969274,2020-12-03 20:38:33
1,1.243127,4.548891e+03,4.684956e+03,75.187792,2020-12-03 21:07:58
2,0.387631,1.461509e+03,1.477470e+03,23.795304,2020-12-04 00:47:41
3,1.502367,5.636909e+03,5.651815e+03,92.000031,2020-12-04 01:22:38
4,1.606812,5.165487e+03,5.196792e+03,91.059361,2020-12-04 04:11:52
...,...,...,...,...,...
209,168.515432,6.041720e+05,6.085494e+05,10000.000000,2021-11-16 11:08:02
210,13.877153,4.975323e+04,5.013791e+04,823.494495,2021-11-16 19:39:52
211,379.159722,1.359387e+06,1.362395e+06,22500.000000,2021-11-17 09:09:30
212,336.900836,1.239267e+06,1.244869e+06,20250.000000,2021-11-19 09:47:17


In [163]:
amzn_ust_df.to_pickle(os.getcwd() + "\\pkl_stories\\amzn_ust_swaps.pkl")
amzn_ust_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\amzn_ust_reserves.pkl")
amzn_ust_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\amzn_ust_mints.pkl")
amzn_ust_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\amzn_ust_burns.pkl")

In [164]:
amzn_ust_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\amzn_ust_swaps.pkl")
amzn_ust_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\amzn_ust_reserves.pkl")
amzn_ust_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\amzn_ust_mints.pkl")
amzn_ust_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\amzn_ust_burns.pkl")

# mBABA/UST (STO)

In [165]:
baba_ust_history = get_pool_v2_history(contract_id='0x676ce85f66adb8d7b8323aeefe17087a3b8cb363', range_limit=3)
all_baba_ust_history, other_baba_ust_history = filter_swaps(baba_ust_history)
baba_ust_df = pool_history_to_df(all_baba_ust_history)
baba_ust_df

100%|██████████| 3/3 [00:03<00:00,  1.15s/it]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,UST,mBABA,950.000000,3.542464,510.868031,2020-12-03 19:03:22,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x443b29e221b54f7be8c0805859f57028cff0a1ca,0xea779b667c8dde199c3abe49b1c732c009276bfb1643...
1,UST,mBABA,1400.000000,5.181502,745.943556,2020-12-03 19:18:51,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xcc05590ba009b10cb30a7b7e87e2f517ea2f4301,0xeefa261f3ca7743fd27ed017e1546cb4d7061f9aec06...
2,mBABA,UST,5.181502,1391.649698,741.772921,2020-12-03 19:22:03,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xcc05590ba009b10cb30a7b7e87e2f517ea2f4301,0xabd1c1da10b4e202e352b92fe858a26f2cc750ca1b08...
3,UST,mBABA,208.000000,0.772721,128.343904,2020-12-03 19:46:39,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x443b29e221b54f7be8c0805859f57028cff0a1ca,0x0bd039d09278d3908f5ceecd06afed1ba1e5fadb9166...
4,UST,mBABA,2468.188484,9.091875,1523.261175,2020-12-03 19:47:40,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xd1373dfb5ff412291c06e5dfe6b25be239dbcf3e,0x1864dddfe597d746b3045dfff96a1db0870eb267c62f...
...,...,...,...,...,...,...,...,...,...
1825,mBABA,UST,300.913685,45541.194822,22727.419478,2021-11-21 06:09:13,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x9fb7e5f71517c9b0059e3200b51bbd1477db333b,0x41568627dc56d5a79aebeec7f28426c874f2c12610ba...
1826,mBABA,UST,201.689323,30120.064750,15193.441629,2021-11-22 14:17:37,0x1111111254fb6c44bac0bed2854e76f90643097d,0x28645d690057b73ac8511ed34d2921992a8085ee,0xc06f4c17739ab535b9459976804df5c27a49af7446b6...
1827,mBABA,UST,204.294479,30184.619032,15196.290163,2021-11-22 20:03:52,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x2c691e3c7c05ebd156e40684ee299f2de62a9fd7,0x7ea19392bed665134439ab5f274420a232b021d85e2a...
1828,mBABA,UST,294.508337,42949.641778,21455.071448,2021-11-23 07:03:22,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x9fb7e5f71517c9b0059e3200b51bbd1477db333b,0x33171a74eb2a12507d42cd6aad27be4e6311815c1bda...


In [166]:
baba_ust_reserves_history = get_pool_v2_reserves_history(contract_id='0x676ce85f66adb8d7b8323aeefe17087a3b8cb363')
baba_ust_reserves_df = pool_reserves_to_df(baba_ust_reserves_history)
baba_ust_reserves_df

100%|██████████| 2/2 [00:00<00:00,  2.64it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,1968.519784,5.234900e+05,5.245581e+05,326.683541,96029.025979,2020-12-03
1,2079.330923,5.591906e+05,5.739436e+05,515.083593,142037.025919,2020-12-04
2,1985.548667,5.547407e+05,5.563241e+05,38.191354,10500.000000,2020-12-05
3,1967.865076,5.597407e+05,5.421465e+05,17.683591,5000.000000,2020-12-06
4,2646.178589,7.017764e+05,7.051406e+05,72.379041,19801.532235,2020-12-07
...,...,...,...,...,...,...
320,37406.587101,5.723803e+06,5.755977e+06,68.754111,10529.971883,2021-11-20
321,37707.500786,5.678262e+06,5.667495e+06,300.913685,45541.194822,2021-11-21
322,38113.484588,5.617957e+06,5.656663e+06,405.983802,60304.683782,2021-11-22
323,38407.992925,5.575007e+06,5.569880e+06,294.508337,42949.641778,2021-11-23


In [167]:
baba_ust_mints_history = get_pool_v2_mints(contract_id='0x676ce85f66adb8d7b8323aeefe17087a3b8cb363', range_limit=2)
baba_ust_mints_df = pool_mints_to_df(baba_ust_mints_history)
baba_ust_mints_df

100%|██████████| 2/2 [00:00<00:00,  2.20it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,1167.962326,311331.417600,0.000000,19068.910998,2020-12-03 16:40:45
1,3.542464,950.040144,1028.415990,58.012517,2020-12-03 19:06:22
2,0.750242,207.513703,256.056507,12.476891,2020-12-03 19:48:08
3,7.805269,2218.220421,2350.304792,131.573890,2020-12-03 19:50:01
4,8.284386,2354.383024,2494.572829,139.650383,2020-12-03 19:50:09
...,...,...,...,...,...
297,142.621975,24768.679900,24878.479098,1861.105498,2021-11-18 05:08:59
298,304.695000,46466.450305,46571.963623,3725.516191,2021-11-19 18:14:51
299,65.745507,10060.108585,10069.258227,805.224934,2021-11-20 12:44:41
300,48.889744,7480.908715,7522.959943,598.782227,2021-11-20 22:03:42


In [168]:
baba_ust_burns_history = get_pool_v2_burns(contract_id='0x676ce85f66adb8d7b8323aeefe17087a3b8cb363', range_limit=2)
baba_ust_burns_df = pool_burns_to_df(baba_ust_burns_history)
baba_ust_burns_df

100%|██████████| 2/2 [00:00<00:00,  2.46it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,31.758278,1.069123e+04,1.207001e+04,582.587828,2020-12-03 20:24:10
1,7.373068,2.349030e+03,2.549442e+03,131.573890,2020-12-03 20:37:06
2,3.975777,1.250350e+03,1.239862e+03,70.489408,2020-12-03 21:53:44
3,6.940986,1.940260e+03,1.955766e+03,115.971999,2020-12-04 03:30:30
4,49.045092,1.362183e+04,1.368730e+04,816.819225,2020-12-04 04:19:21
...,...,...,...,...,...
138,7723.419893,1.320394e+06,1.323086e+06,100000.000000,2021-11-10 10:53:27
139,50.075941,8.685675e+03,8.672624e+03,653.062853,2021-11-12 10:48:02
140,3064.085471,5.325425e+05,5.362295e+05,40000.000000,2021-11-16 11:10:06
141,7851.719019,1.364640e+06,1.367392e+06,102500.000000,2021-11-17 09:11:00


In [169]:
baba_ust_df.to_pickle(os.getcwd() + "\\pkl_stories\\baba_ust_swaps.pkl")
baba_ust_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\baba_ust_reserves.pkl")
baba_ust_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\baba_ust_mints.pkl")
baba_ust_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\baba_ust_burns.pkl")

In [170]:
baba_ust_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\baba_ust_swaps.pkl")
baba_ust_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\baba_ust_reserves.pkl")
baba_ust_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\baba_ust_mints.pkl")
baba_ust_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\baba_ust_burns.pkl")

# mAPPLE/UST (STO)

In [172]:
apple_ust_history = get_pool_v2_history(contract_id='0xb022e08adc8ba2de6ba4fecb59c6d502f66e953b', range_limit=4)
all_apple_ust_history, other_apple_ust_history = filter_swaps(apple_ust_history)
apple_ust_df = pool_history_to_df(all_apple_ust_history)
apple_ust_df

100%|██████████| 4/4 [00:03<00:00,  1.18it/s]

list index out of range


,token_in,token_out,amount_in,amount_out,amount_usd,timestamp,sender,to,txd
0,UST,mAAPL,297.072343,2.404518,149.249384,2020-12-03 17:54:40,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x8f70b3ac45a6896532fb90b992d5b7827ba88d3c,0x5c450e4f9908aa58814be20fc50d60b03c3798f99af7...
1,UST,mAAPL,1000.000000,8.060584,531.155662,2020-12-03 19:25:57,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x55e1490a1878d0b61811726e2cb96560022e764c,0x5cc783f79cdab6a966cc9de5d69215dd4199c1f367af...
2,UST,mAAPL,1000.000000,8.009403,529.764717,2020-12-03 19:50:13,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xc5cee9416571752528acbb4331761c5e2706748c,0x0d5279feeb6c1536f2f8ee5efda3901352e6697fb84a...
3,UST,mAAPL,10000.000000,77.390496,5504.266915,2020-12-03 19:51:07,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x36d2269af2b297ca11b1c61e8a3baf140b177ea1,0x339975916e81727fbf0060810da871921ca5aecb85aa...
4,UST,mAAPL,5000.000000,36.938175,2752.007649,2020-12-03 19:52:27,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x55e1490a1878d0b61811726e2cb96560022e764c,0xd2ad1e3793ea315c70b1bd90999caffbe498eb5aefab...
...,...,...,...,...,...,...,...,...,...
2112,mAAPL,UST,249.750001,40532.477766,20412.497099,2021-11-23 18:56:37,0x1111111254fb6c44bac0bed2854e76f90643097d,0x5690df030163680f4da2046798614c3f5c24f64a,0x7d519aeb1052866c81c15ddcc0eb3e8bdeda5b4a7579...
2113,mAAPL,UST,102.461207,16498.921790,8285.772340,2021-11-24 04:46:57,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x64bade3ea5ffc64cfb776f55cf82086b9429a5ee,0xafce0f19611c4fa06045a8561514dd7179261ccba795...
2114,mAAPL,UST,181.894384,29107.022513,14635.546498,2021-11-24 06:41:41,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x96fd33b5f98a2ff69189e5543850117922c4f23b,0x0c88da6763d4694544fbc99c2298fb8ec5f424c83d92...
2115,UST,mAAPL,35000.000000,217.240597,17598.702106,2021-11-24 06:41:57,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x9fb7e5f71517c9b0059e3200b51bbd1477db333b,0xde1d9e9b7ffa66010e2e23a548f3321259b16c16d6cf...


In [173]:
apple_ust_reserves_history = get_pool_v2_reserves_history(contract_id='0xb022e08adc8ba2de6ba4fecb59c6d502f66e953b')
apple_ust_reserves_df = pool_reserves_to_df(apple_ust_reserves_history)
apple_ust_reserves_df

100%|██████████| 2/2 [00:00<00:00,  2.68it/s]

list index out of range


,reserve0,reserve1,reserveUSD,dailyVolumeToken0,dailyVolumeToken1,date
0,1.401786e+06,11264.427046,1.401069e+06,79345.668759,613.107702,2020-12-03
1,2.380533e+06,19116.460462,2.388652e+06,109925.602519,886.243104,2020-12-04
2,2.400318e+06,18972.563025,2.526287e+06,49249.641918,394.007029,2020-12-05
3,2.508393e+06,18791.708387,2.506214e+06,117349.571073,893.440321,2020-12-06
4,2.396930e+06,19453.772147,2.407887e+06,128500.441615,1013.195143,2020-12-07
...,...,...,...,...,...,...
332,7.285191e+06,44949.125059,7.290475e+06,59483.256071,368.721740,2021-11-20
333,7.285285e+06,44949.125059,7.263000e+06,47.000632,0.289119,2021-11-21
334,7.325488e+06,44704.662446,7.367235e+06,40081.528156,245.210162,2021-11-22
335,7.335508e+06,45315.746248,7.370754e+06,102798.912472,631.644385,2021-11-23


In [174]:
apple_ust_mints_history = get_pool_v2_mints(contract_id='0xb022e08adc8ba2de6ba4fecb59c6d502f66e953b', range_limit=2)
apple_ust_mints_df = pool_mints_to_df(apple_ust_mints_history)
apple_ust_mints_df

100%|██████████| 2/2 [00:01<00:00,  1.92it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,311664.217000,2532.620000,0.000000,28094.964482,2020-12-03 15:39:17
1,61.647178,0.500000,64.533043,5.551892,2020-12-03 18:38:02
2,1000.195278,8.060584,1147.152126,89.788960,2020-12-03 19:28:29
3,5061.686850,36.938175,5553.016991,432.365141,2020-12-03 19:53:53
4,1051.869104,7.408828,1175.173476,88.269255,2020-12-03 19:55:33
...,...,...,...,...,...
510,121.165592,0.747549,121.265654,9.424242,2021-11-22 11:22:20
511,23966.256733,146.256942,23945.672571,1853.923501,2021-11-23 05:51:12
512,228781.677737,1409.092847,229122.530817,17779.145523,2021-11-23 09:51:14
513,64259.910000,396.971243,64568.669355,5001.194757,2021-11-23 22:16:05


In [175]:
apple_ust_burns_history = get_pool_v2_burns(contract_id='0xb022e08adc8ba2de6ba4fecb59c6d502f66e953b', range_limit=2)
apple_ust_burns_df = pool_burns_to_df(apple_ust_burns_history)
apple_ust_burns_df

100%|██████████| 2/2 [00:00<00:00,  2.46it/s]

list index out of range


,amount0,amount1,amountUSD,liquidity,timestamp
0,9.257404e+03,72.808590,9.288770e+03,820.734666,2020-12-04 01:38:11
1,2.680000e+02,2.125649,2.708712e+02,23.860217,2020-12-04 04:45:32
2,1.747303e+04,138.674535,1.771346e+04,1556.121518,2020-12-04 06:46:27
3,8.613714e+03,69.368169,8.652346e+03,772.736781,2020-12-04 13:09:54
4,1.797690e+04,145.813180,1.800050e+04,1618.488751,2020-12-04 16:28:25
...,...,...,...,...,...
268,1.282434e+06,8353.332493,1.283040e+06,102500.000000,2021-11-17 08:54:01
269,1.178052e+06,7366.755869,1.182610e+06,92250.000000,2021-11-19 09:44:06
270,2.488983e+04,153.941644,2.505837e+04,1938.346285,2021-11-20 00:18:59
271,2.636725e+05,1609.092847,2.640339e+05,20396.536432,2021-11-23 09:41:38


In [176]:
apple_ust_df.to_pickle(os.getcwd() + "\\pkl_stories\\apple_ust_swaps.pkl")
apple_ust_reserves_df.to_pickle(os.getcwd() + "\\pkl_stories\\apple_ust_reserves.pkl")
apple_ust_mints_df.to_pickle(os.getcwd() + "\\pkl_stories\\apple_ust_mints.pkl")
apple_ust_burns_df.to_pickle(os.getcwd() + "\\pkl_stories\\apple_ust_burns.pkl")

In [177]:
apple_ust_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\apple_ust_swaps.pkl")
apple_ust_reserves_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\apple_ust_reserves.pkl")
apple_ust_mints_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\apple_ust_mints.pkl")
apple_ust_burns_df = pd.read_pickle(os.getcwd() + "\\pkl_stories\\apple_ust_burns.pkl")

# Writing some specific data

In [205]:
# other_wbtc_usdc_history         - this one is empty
print(len(other_weth_usdc_history))
# other_wbtc_dai_history          - this one is empty
# other_alice_weth_history        - this one is empty
# other_apple_ust_history         - this one is empty
# other_amzn_ust_history          - this one is empty
# other_baba_ust_history          - this one is empty
print(len(other_axs_weth_v2_history))
# other_bpt_weth_history          - this one is empty
# other_doge_weth_history         - this one is empty
print(len(other_elon_weth_history))
# other_enj_eth_history           - this one is empty
print(len(other_fei_weth_history))
# other_ixs_weth_history          - this one is empty
print(len(other_mana_weth_history))
# other_perl_weth_history         - this one is empty
print(len(other_sand_weth_history))
print(len(other_shib_weth_history))
print(len(other_squid_weth_history))
# other_stonks_usdc_history       - this one is empty
# other_uma_fei_history           - this one is empty
# other_xaut_weth_history         - this one is empty
# other_hkmt_usdt_history         - this one is empty


74
2
2
2
1
6
3
149


In [212]:
other_weth_usdc_df = pool_history_to_df(other_weth_usdc_history)
other_axs_weth_v2_df = pool_history_to_df(other_axs_weth_v2_history)
other_elon_weth_df = pool_history_to_df(other_elon_weth_history)
other_fei_weth_df = pool_history_to_df(other_fei_weth_history)
other_mana_weth_df = pool_history_to_df(other_mana_weth_history)
other_sand_weth_df = pool_history_to_df(other_sand_weth_history)
other_shib_weth_df = pool_history_to_df(other_shib_weth_history)
other_squid_weth_df = pool_history_to_df(other_squid_weth_history)


In [213]:
other_weth_usdc_df.to_pickle(os.getcwd() + "\\pkl_stories\\other_weth_usdc_swaps.pkl")
other_axs_weth_v2_df.to_pickle(os.getcwd() + "\\pkl_stories\\other_axs_weth_v2_swaps.pkl")
other_elon_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\other_elon_weth_swaps.pkl")
other_fei_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\other_fei_weth_swaps.pkl")
other_mana_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\other_mana_weth_swaps.pkl")
other_sand_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\other_sand_weth_swaps.pkl")
other_shib_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\other_shib_weth_swaps.pkl")
other_squid_weth_df.to_pickle(os.getcwd() + "\\pkl_stories\\other_squid_weth_swaps.pkl")
